# LIBRARY

In [ ]:
# Untuk keperluan Database
import sqlite3
import uuid
from datetime import datetime

#Untuk keperluan model sistem rekomendasi
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# Untuk kuisioner
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Tampilkan semua baris
pd.set_option('display.max_rows', None)
# Tampilkan semua kolom
pd.set_option('display.max_columns', None)
# Tampilkan isi sel sepanjang apapun
pd.set_option('display.max_colwidth', None)
# Jangan potong tampilan secara horizontal
pd.set_option('display.expand_frame_repr', True)
# Lebarkan tampilan ke batas maksimum
pd.set_option('display.width', 1000)

# VIEW DATABASE

In [ ]:
# View database
db_path = "/content/bone_recommender.db"

# Connect ke database
conn = sqlite3.connect(db_path)

# Baca tabel ke DataFrame
df = pd.read_sql_query("SELECT * FROM mst_tbl", conn)

# Tampilkan sebagai tabel
df

,user_id,created_at,kategori,umur,gender,domisili,INN-CE1,SE-M1,NACH-FF2,LOC-I1,SE-P2,INN-O2,SE-IP1,NACH-HS1,SE-IF1,LOC-E2,SE-S1,INN-W1,NACH-HS2,SE-IF2,INN-CE2,LOC-E1,SE-M2,SE-S2,LOC-I2,INN-W2,SE-P1,INN-O1,SE-IP2,NACH-FF1,CON-2,OPE-3,NEU-1,AGR-2,EXT-2,CON-1,OPE-1,AGR-1,NEU-3,EXT-3,OPE-2,AGR-3,NEU-2,CON-3,EXT-1
0,89a5a8ff-b48a-405a-bd2a-efb9e1423526,2025-12-29 15:42:08.177049,Calon Wirausahawan,18,Laki-laki,Kabupaten Aceh Barat,5,5,4,4,5,5,4,4,5,5,4,4,5,5,4,4,5,5,4,4,5,5,4,4,5,4,4,5,4,3,4,5,4,3,4,5,4,3,5
1,ef491c40-e42d-47ca-92c6-33836869aaab,2025-12-29 16:22:26.476166,Calon Wirausahawan,18,Laki-laki,Kabupaten Aceh Barat,5,3,3,4,3,4,4,5,5,4,4,5,3,4,4,3,4,3,4,4,4,3,4,4,3,5,4,4,5,3,4,3,5,4,4,5,4,5,5


In [ ]:
# # setiap selesai submit, db harus di close
# conn.close()

# CONNECT & CREATE DATABASE

In [ ]:
# Koneksi ke database setiap mulai sistem
conn = sqlite3.connect("/content/bone_recommender.db")
cursor = conn.cursor()

In [ ]:
cursor.execute("""
create table if not exists mst_tbl(
user_id TEXT PRIMARY KEY,
created_at TEXT,
kategori TEXT,
umur INT,
gender TEXT,
domisili TEXT,


'INN-CE1' INT, 'SE-M1' INT,
'NACH-FF2' INT, 'LOC-I1' INT,
'SE-P2' INT, 'INN-O2' INT,
'SE-IP1' INT, 'NACH-HS1' INT,
'SE-IF1' INT, 'LOC-E2' INT,
'SE-S1' INT, 'INN-W1' INT,
'NACH-HS2' INT, 'SE-IF2' INT,
'INN-CE2' INT, 'LOC-E1' INT,
'SE-M2' INT, 'SE-S2' INT,
'LOC-I2' INT, 'INN-W2' INT,
'SE-P1' INT, 'INN-O1' INT,
'SE-IP2' INT, 'NACH-FF1' INT,


'CON-2' INT, 'OPE-3' INT,
'NEU-1' INT, 'AGR-2' INT,
'EXT-2' INT, 'CON-1' INT,
'OPE-1' INT, 'AGR-1' INT,
'NEU-3' INT, 'EXT-3' INT,
'OPE-2' INT, 'AGR-3' INT,
'NEU-2' INT, 'CON-3' INT,
'EXT-1' INT
)
""")

# BIODATA (INPUT)


In [ ]:
# KATEGORI
kategori_label = widgets.Label(
    "Mohon pilih kategori yang paling menggambarkan peran Anda dalam aktivitas ekonomi atau profesional saat ini:"
)
kategori_input = widgets.Dropdown(
    options=["Calon Wirausahawan", "Pelaku UMKM", "Pelajar/Mahasiswa", "Profesional"],
    layout=widgets.Layout(width='350px')
)
kategori_block = widgets.VBox([kategori_label, kategori_input])

# UMUR
umur_label = widgets.Label("Berapa usia Anda saat ini?")
umur_input = widgets.BoundedIntText(
    value=18,
    min=1,
    max=99,
    layout=widgets.Layout(width='120px')
)
umur_block = widgets.VBox([umur_label, umur_input])

# GENDER
gender_label = widgets.Label("Jenis kelamin Anda:")
gender_input = widgets.Dropdown(
    options=["Laki-laki", "Perempuan"],
    layout=widgets.Layout(width='200px')
)
gender_block = widgets.VBox([gender_label, gender_input])

# DOMISILI
domisili_label = widgets.Label("Dimana domisili Anda saat ini?")

domisili_input = widgets.Combobox(
    options=[
          # ACEH
          "Kabupaten Aceh Barat", "Kabupaten Aceh Barat Daya", "Kabupaten Aceh Besar",
          "Kabupaten Aceh Jaya", "Kabupaten Aceh Selatan", "Kabupaten Aceh Singkil",
          "Kabupaten Aceh Tamiang", "Kabupaten Aceh Tengah", "Kabupaten Aceh Tenggara",
          "Kabupaten Aceh Timur", "Kabupaten Aceh Utara", "Kabupaten Bener Meriah",
          "Kabupaten Bireuen", "Kabupaten Gayo Lues", "Kabupaten Nagan Raya",
          "Kabupaten Pidie", "Kabupaten Pidie Jaya", "Kabupaten Simeulue",
          "Kota Banda Aceh", "Kota Langsa", "Kota Lhokseumawe", "Kota Sabang",
          "Kota Subulussalam",

          # SUMATERA UTARA
          "Kabupaten Asahan", "Kabupaten Batubara", "Kabupaten Dairi", "Kabupaten Deli Serdang",
          "Kabupaten Humbang Hasundutan", "Kabupaten Karo", "Kabupaten Labuhanbatu",
          "Kabupaten Labuhanbatu Selatan", "Kabupaten Labuhanbatu Utara", "Kabupaten Langkat",
          "Kabupaten Mandailing Natal", "Kabupaten Nias", "Kabupaten Nias Barat",
          "Kabupaten Nias Selatan", "Kabupaten Nias Utara", "Kabupaten Padang Lawas",
          "Kabupaten Padang Lawas Utara", "Kabupaten Pakpak Bharat", "Kabupaten Samosir",
          "Kabupaten Serdang Bedagai", "Kabupaten Simalungun", "Kabupaten Tapanuli Selatan",
          "Kabupaten Tapanuli Tengah", "Kabupaten Tapanuli Utara", "Kabupaten Toba",
          "Kota Binjai", "Kota Gunungsitoli", "Kota Medan", "Kota Padangsidimpuan",
          "Kota Pematangsiantar", "Kota Sibolga", "Kota Tanjungbalai", "Kota Tebing Tinggi",

          # SUMATERA BARAT
          "Kabupaten Agam", "Kabupaten Dharmasraya", "Kabupaten Kepulauan Mentawai",
          "Kabupaten Lima Puluh Kota", "Kabupaten Padang Pariaman", "Kabupaten Pasaman",
          "Kabupaten Pasaman Barat", "Kabupaten Pesisir Selatan", "Kabupaten Sijunjung",
          "Kabupaten Solok", "Kabupaten Solok Selatan", "Kabupaten Tanah Datar",
          "Kota Bukittinggi", "Kota Padang", "Kota Padangpanjang", "Kota Pariaman",
          "Kota Payakumbuh", "Kota Sawahlunto", "Kota Solok",

          # RIAU
          "Kabupaten Bengkalis", "Kabupaten Indragiri Hilir", "Kabupaten Indragiri Hulu",
          "Kabupaten Kampar", "Kabupaten Kepulauan Meranti", "Kabupaten Kuantan Singingi",
          "Kabupaten Pelalawan", "Kabupaten Rokan Hilir", "Kabupaten Rokan Hulu",
          "Kabupaten Siak", "Kota Dumai", "Kota Pekanbaru",

          # JAMBI
          "Kabupaten Batanghari", "Kabupaten Bungo", "Kabupaten Kerinci",
          "Kabupaten Merangin", "Kabupaten Muaro Jambi", "Kabupaten Sarolangun",
          "Kabupaten Tanjung Jabung Barat", "Kabupaten Tanjung Jabung Timur",
          "Kabupaten Tebo", "Kota Jambi", "Kota Sungai Penuh",

          # SUMATERA SELATAN
          "Kabupaten Banyuasin", "Kabupaten Empat Lawang", "Kabupaten Lahat",
          "Kabupaten Muara Enim", "Kabupaten Musi Banyuasin", "Kabupaten Musi Rawas",
          "Kabupaten Musi Rawas Utara", "Kabupaten Ogan Ilir", "Kabupaten Ogan Komering Ilir",
          "Kabupaten Ogan Komering Ulu", "Kabupaten Ogan Komering Ulu Selatan",
          "Kabupaten Ogan Komering Ulu Timur", "Kota Lubuklinggau", "Kota Pagar Alam",
          "Kota Palembang", "Kota Prabumulih",

          # BENGKULU
          "Kabupaten Bengkulu Selatan", "Kabupaten Bengkulu Tengah", "Kabupaten Bengkulu Utara",
          "Kabupaten Kaur", "Kabupaten Kepahiang", "Kabupaten Lebong",
          "Kabupaten Mukomuko", "Kabupaten Rejang Lebong", "Kabupaten Seluma",
          "Kota Bengkulu",

          # LAMPUNG
          "Kabupaten Lampung Barat", "Kabupaten Lampung Selatan", "Kabupaten Lampung Tengah",
          "Kabupaten Lampung Timur", "Kabupaten Lampung Utara", "Kabupaten Mesuji",
          "Kabupaten Pesawaran", "Kabupaten Pesisir Barat", "Kabupaten Pringsewu",
          "Kabupaten Tanggamus", "Kabupaten Tulang Bawang", "Kabupaten Tulang Bawang Barat",
          "Kabupaten Way Kanan", "Kota Bandar Lampung", "Kota Metro",

          # KEP. BANGKA BELITUNG
          "Kabupaten Bangka", "Kabupaten Bangka Barat", "Kabupaten Bangka Selatan",
          "Kabupaten Bangka Tengah", "Kabupaten Belitung", "Kabupaten Belitung Timur",
          "Kota Pangkalpinang",

          # KEP. RIAU
          "Kabupaten Bintan", "Kabupaten Karimun", "Kabupaten Kepulauan Anambas",
          "Kabupaten Lingga", "Kabupaten Natuna", "Kota Batam", "Kota Tanjungpinang",

          # DKI JAKARTA
          "Kota Jakarta Barat", "Kota Jakarta Pusat", "Kota Jakarta Selatan",
          "Kota Jakarta Timur", "Kota Jakarta Utara", "Kabupaten Kepulauan Seribu",

          # JAWA BARAT
          "Kabupaten Bandung", "Kabupaten Bandung Barat", "Kabupaten Bekasi",
          "Kabupaten Bogor", "Kabupaten Ciamis", "Kabupaten Cianjur", "Kabupaten Cirebon",
          "Kabupaten Garut", "Kabupaten Indramayu", "Kabupaten Karawang",
          "Kabupaten Kuningan", "Kabupaten Majalengka", "Kabupaten Pangandaran",
          "Kabupaten Purwakarta", "Kabupaten Subang", "Kabupaten Sukabumi",
          "Kabupaten Sumedang", "Kabupaten Tasikmalaya", "Kota Bandung", "Kota Banjar",
          "Kota Bekasi", "Kota Bogor", "Kota Cimahi", "Kota Cirebon", "Kota Depok",
          "Kota Sukabumi", "Kota Tasikmalaya",

          # JAWA TENGAH
          "Kabupaten Banjarnegara", "Kabupaten Banyumas", "Kabupaten Batang",
          "Kabupaten Blora", "Kabupaten Boyolali", "Kabupaten Brebes", "Kabupaten Cilacap",
          "Kabupaten Demak", "Kabupaten Grobogan", "Kabupaten Jepara",
          "Kabupaten Karanganyar", "Kabupaten Kebumen", "Kabupaten Kendal",
          "Kabupaten Klaten", "Kabupaten Kudus", "Kabupaten Magelang",
          "Kabupaten Pati", "Kabupaten Pekalongan", "Kabupaten Pemalang",
          "Kabupaten Purbalingga", "Kabupaten Purworejo", "Kabupaten Rembang",
          "Kabupaten Semarang", "Kabupaten Sragen", "Kabupaten Sukoharjo",
          "Kabupaten Tegal", "Kabupaten Temanggung", "Kabupaten Wonogiri",
          "Kabupaten Wonosobo", "Kota Magelang", "Kota Pekalongan", "Kota Salatiga",
          "Kota Semarang", "Kota Surakarta", "Kota Tegal",

          # DI YOGYAKARTA
          "Kabupaten Bantul", "Kabupaten Gunungkidul", "Kabupaten Kulon Progo",
          "Kabupaten Sleman", "Kota Yogyakarta",

          # JAWA TIMUR
          "Kabupaten Bangkalan", "Kabupaten Banyuwangi", "Kabupaten Blitar",
          "Kabupaten Bojonegoro", "Kabupaten Bondowoso", "Kabupaten Gresik",
          "Kabupaten Jember", "Kabupaten Jombang", "Kabupaten Kediri",
          "Kabupaten Lamongan", "Kabupaten Lumajang", "Kabupaten Madiun",
          "Kabupaten Magetan", "Kabupaten Malang", "Kabupaten Mojokerto",
          "Kabupaten Nganjuk", "Kabupaten Ngawi", "Kabupaten Pacitan",
          "Kabupaten Pamekasan", "Kabupaten Pasuruan", "Kabupaten Ponorogo",
          "Kabupaten Probolinggo", "Kabupaten Sampang", "Kabupaten Sidoarjo",
          "Kabupaten Situbondo", "Kabupaten Sumenep", "Kabupaten Trenggalek",
          "Kabupaten Tuban", "Kabupaten Tulungagung", "Kota Batu", "Kota Blitar",
          "Kota Kediri", "Kota Madiun", "Kota Malang", "Kota Mojokerto",
          "Kota Pasuruan", "Kota Probolinggo", "Kota Surabaya",

          # BANTEN
          "Kabupaten Lebak", "Kabupaten Pandeglang",
          "Kabupaten Serang", "Kabupaten Tangerang",
          "Kota Cilegon", "Kota Serang", "Kota Tangerang",
          "Kota Tangerang Selatan",

          # BALI
          "Kabupaten Badung", "Kabupaten Bangli", "Kabupaten Buleleng",
          "Kabupaten Gianyar", "Kabupaten Jembrana", "Kabupaten Karangasem",
          "Kabupaten Klungkung", "Kabupaten Tabanan", "Kota Denpasar",

          # NUSA TENGGARA BARAT
          "Kabupaten Bima", "Kabupaten Dompu", "Kabupaten Lombok Barat",
          "Kabupaten Lombok Tengah", "Kabupaten Lombok Timur", "Kabupaten Lombok Utara",
          "Kabupaten Sumbawa", "Kabupaten Sumbawa Barat", "Kota Bima", "Kota Mataram",

          # NUSA TENGGARA TIMUR
          "Kabupaten Alor", "Kabupaten Belu", "Kabupaten Ende", "Kabupaten Flores Timur",
          "Kabupaten Kupang", "Kabupaten Lembata", "Kabupaten Malaka", "Kabupaten Manggarai",
          "Kabupaten Manggarai Barat", "Kabupaten Manggarai Timur", "Kabupaten Ngada",
          "Kabupaten Nagekeo", "Kabupaten Rote Ndao", "Kabupaten Sabu Raijua",
          "Kabupaten Sikka", "Kabupaten Sumba Barat", "Kabupaten Sumba Barat Daya",
          "Kabupaten Sumba Tengah", "Kabupaten Sumba Timur", "Kabupaten Timor Tengah Selatan",
          "Kabupaten Timor Tengah Utara", "Kota Kupang",

          # KALIMANTAN BARAT
          "Kabupaten Bengkayang", "Kabupaten Kapuas Hulu", "Kabupaten Kayong Utara",
          "Kabupaten Ketapang", "Kabupaten Kubu Raya", "Kabupaten Landak",
          "Kabupaten Melawi", "Kabupaten Mempawah", "Kabupaten Sambas",
          "Kabupaten Sanggau", "Kabupaten Sekadau", "Kabupaten Sintang",
          "Kota Pontianak", "Kota Singkawang",

          # KALIMANTAN TENGAH
          "Kabupaten Barito Selatan", "Kabupaten Barito Timur", "Kabupaten Barito Utara",
          "Kabupaten Gunung Mas", "Kabupaten Kapuas", "Kabupaten Katingan",
          "Kabupaten Kotawaringin Barat", "Kabupaten Kotawaringin Timur",
          "Kabupaten Lamandau", "Kabupaten Murung Raya", "Kabupaten Pulang Pisau",
          "Kabupaten Seruyan", "Kabupaten Sukamara", "Kota Palangka Raya",

          # KALIMANTAN SELATAN
          "Kabupaten Balangan", "Kabupaten Banjar", "Kabupaten Barito Kuala",
          "Kabupaten Hulu Sungai Selatan", "Kabupaten Hulu Sungai Tengah",
          "Kabupaten Hulu Sungai Utara", "Kabupaten Kotabaru", "Kabupaten Tabalong",
          "Kabupaten Tanah Bumbu", "Kabupaten Tanah Laut", "Kabupaten Tapin",
          "Kota Banjarbaru", "Kota Banjarmasin",

          # KALIMANTAN TIMUR
          "Kabupaten Berau", "Kabupaten Kutai Barat", "Kabupaten Kutai Kartanegara",
          "Kabupaten Kutai Timur", "Kabupaten Mahakam Ulu", "Kabupaten Paser",
          "Kabupaten Penajam Paser Utara", "Kota Balikpapan", "Kota Bontang",
          "Kota Samarinda",

          # KALIMANTAN UTARA
          "Kabupaten Bulungan", "Kabupaten Malinau", "Kabupaten Nunukan",
          "Kabupaten Tana Tidung", "Kota Tarakan",

          # SULAWESI UTARA
          "Kabupaten Bolaang Mongondow", "Kabupaten Bolaang Mongondow Selatan",
          "Kabupaten Bolaang Mongondow Timur", "Kabupaten Bolaang Mongondow Utara",
          "Kabupaten Kepulauan Sangihe", "Kabupaten Kepulauan Siau Tagulandang Biaro",
          "Kabupaten Kepulauan Talaud", "Kabupaten Minahasa", "Kabupaten Minahasa Selatan",
          "Kabupaten Minahasa Tenggara", "Kabupaten Minahasa Utara",
          "Kota Bitung", "Kota Kotamobagu", "Kota Manado", "Kota Tomohon",

          # SULAWESI TENGAH
          "Kabupaten Banggai", "Kabupaten Banggai Kepulauan", "Kabupaten Banggai Laut",
          "Kabupaten Buol", "Kabupaten Donggala", "Kabupaten Morowali",
          "Kabupaten Morowali Utara", "Kabupaten Parigi Moutong", "Kabupaten Poso",
          "Kabupaten Sigi", "Kabupaten Tojo Una-Una", "Kabupaten Tolitoli",
          "Kota Palu",

          # SULAWESI SELATAN
          "Kabupaten Bantaeng", "Kabupaten Barru", "Kabupaten Bone", "Kabupaten Bulukumba",
          "Kabupaten Enrekang", "Kabupaten Gowa", "Kabupaten Jeneponto",
          "Kabupaten Kepulauan Selayar", "Kabupaten Luwu", "Kabupaten Luwu Timur",
          "Kabupaten Luwu Utara", "Kabupaten Maros", "Kabupaten Pangkajene Kepulauan",
          "Kabupaten Pinrang", "Kabupaten Sidenreng Rappang", "Kabupaten Sinjai",
          "Kabupaten Soppeng", "Kabupaten Takalar", "Kabupaten Tana Toraja",
          "Kabupaten Toraja Utara", "Kota Makassar", "Kota Palopo", "Kota Parepare",

          # SULAWESI TENGGARA
          "Kabupaten Bombana", "Kabupaten Buton", "Kabupaten Buton Selatan",
          "Kabupaten Buton Tengah", "Kabupaten Buton Utara", "Kabupaten Kolaka",
          "Kabupaten Kolaka Timur", "Kabupaten Kolaka Utara", "Kabupaten Konawe",
          "Kabupaten Konawe Kepulauan", "Kabupaten Konawe Selatan",
          "Kabupaten Konawe Utara", "Kabupaten Muna", "Kabupaten Muna Barat",
          "Kabupaten Wakatobi", "Kota Baubau", "Kota Kendari",

          # GORONTALO
          "Kabupaten Boalemo", "Kabupaten Bone Bolango", "Kabupaten Gorontalo",
          "Kabupaten Gorontalo Utara", "Kabupaten Pohuwato", "Kota Gorontalo",

          # SULAWESI BARAT
          "Kabupaten Majene", "Kabupaten Mamasa", "Kabupaten Mamuju",
          "Kabupaten Mamuju Tengah", "Kabupaten Pasangkayu", "Kabupaten Polewali Mandar",

          # MALUKU
          "Kabupaten Buru", "Kabupaten Buru Selatan", "Kabupaten Kepulauan Aru",
          "Kabupaten Maluku Barat Daya", "Kabupaten Maluku Tengah",
          "Kabupaten Maluku Tenggara", "Kabupaten Seram Bagian Barat",
          "Kabupaten Seram Bagian Timur", "Kota Ambon", "Kota Tual",

          # MALUKU UTARA
          "Kabupaten Halmahera Barat", "Kabupaten Halmahera Tengah",
          "Kabupaten Halmahera Timur", "Kabupaten Halmahera Selatan",
          "Kabupaten Halmahera Utara", "Kabupaten Kepulauan Sula",
          "Kabupaten Pulau Morotai", "Kabupaten Pulau Taliabu",
          "Kota Ternate", "Kota Tidore Kepulauan",

          # PAPUA (lama)
          "Kabupaten Asmat", "Kabupaten Biak Numfor", "Kabupaten Boven Digoel",
          "Kabupaten Deiyai", "Kabupaten Dogiyai", "Kabupaten Intan Jaya",
          "Kabupaten Jayapura", "Kabupaten Jayawijaya", "Kabupaten Keerom",
          "Kabupaten Lanny Jaya", "Kabupaten Mamberamo Raya", "Kabupaten Mamberamo Tengah",
          "Kabupaten Mappi", "Kabupaten Merauke", "Kabupaten Mimika",
          "Kabupaten Nabire", "Kabupaten Nduga", "Kabupaten Paniai",
          "Kabupaten Pegunungan Bintang", "Kabupaten Puncak", "Kabupaten Puncak Jaya",
          "Kabupaten Sarmi", "Kabupaten Supiori", "Kabupaten Tolikara",
          "Kabupaten Waropen", "Kota Jayapura",

          # PAPUA TENGAH
          "Kabupaten Puncak", "Kabupaten Paniai", "Kabupaten Dogiyai", "Kabupaten Deiyai",
          "Kabupaten Nabire", "Kabupaten Mimika",

          # PAPUA PEGUNUNGAN
          "Kabupaten Jayawijaya", "Kabupaten Lanny Jaya", "Kabupaten Mamberamo Tengah",
          "Kabupaten Nduga", "Kabupaten Tolikara", "Kabupaten Yahukimo",
          "Kabupaten Yalimo", "Kabupaten Pegunungan Bintang",

          # PAPUA SELATAN
          "Kabupaten Merauke", "Kabupaten Mappi", "Kabupaten Asmat",
          "Kabupaten Boven Digoel",

          # PAPUA BARAT
          "Kabupaten Fakfak", "Kabupaten Kaimana", "Kabupaten Manokwari",
          "Kabupaten Manokwari Selatan", "Kabupaten Pegunungan Arfak",
          "Kabupaten Teluk Bintuni", "Kabupaten Teluk Wondama",
          "Kota Sorong",

          # PAPUA BARAT DAYA (baru)
          "Kabupaten Sorong", "Kabupaten Sorong Selatan", "Kabupaten Tambrauw",
          "Kabupaten Maybrat", "Kabupaten Raja Ampat", "Kota Sorong"
      ],
    placeholder='Ketik untuk mencari atau masukkan sendiri...',
    ensure_option=False,             # supaya input manual tetap diterima
    layout=widgets.Layout(width='350px')
)

# Gabungkan tampilan
domisili_block = widgets.VBox([domisili_label, domisili_input])

In [ ]:
display(kategori_block, umur_block, gender_block, domisili_block)

# DOT FUNCTION (UI KUESIONER)

In [ ]:
def likert_item():
    left_label = widgets.Label("Sangat Tidak Setuju", layout=widgets.Layout(width="150px"))
    right_label = widgets.Label("Sangat Setuju", layout=widgets.Layout(width="150px"))

    scale = widgets.ToggleButtons(
        options=[1, 2, 3, 4, 5],
        button_style='',   # default
        tooltips=['1','2','3','4','5'],
        layout=widgets.Layout(width='250px')
    )

    # sedikit styling agar kelihatan bulat
    scale.style.button_width = '40px'

    likert_row = widgets.HBox([
        left_label,
        scale,
        right_label
    ])

    return scale, likert_row

# BAGIAN A (INPUT)

In [ ]:
questions_A = [
    "Saya tertantang oleh ketidakpastian dan masalah yang belum terpecahkan. (INN-CE1)",
    "Saya percaya diri dapat melakukan networking, yaitu membangun hubungan dan bertukar informasi yang bermanfaat dengan berbagai pihak untuk mendukung kegiatan atau proyek saya. (SE-M1)",
    "Jika saya tidak langsung mengerti sebuah masalah, saya mulai merasa cemas. (NACH-FF2)",
    "Jika saya bekerja dengan sungguh-sungguh, saya bisa mencapai hasil yang saya inginkan. (LOC-I1)",
    "Saya percaya diri dapat memperkirakan jumlah modal awal dan modal kerja yang diperlukan untuk memulai atau menjalankan usaha, proyek, atau kegiatan baru. (SE-P2)",
    "Saya terbuka untuk menggunakan cara baru meskipun belum umum dipraktikkan orang lain. (INN-O2)",
    "Saya percaya diri dapat memotivasi dan mendorong anggota tim agar semangat dalam bekerja. (SE-IP1)",
    "Saya termotivasi untuk segera bertindak ketika menghadapi tantangan yang bisa saya selesaikan. (NACH-HS1)",
    "Saya percaya diri dapat mengorganisir dan memelihara catatan keuangan untuk usaha, proyek, atau kegiatan yang saya jalankan. (SE-IF1)",
    "Saya percaya bahwa hasil kerja saya banyak dipengaruhi oleh keadaan atau orang lain, bukan sepenuhnya oleh usaha saya. (LOC-E2)",
    "Saya percaya diri dapat mengidentifikasi kebutuhan atau peluang baru yang bisa diwujudkan menjadi produk, layanan, atau solusi. (SE-S1)",
    "Saya bersedia untuk mencoba cara baru meskipun berbeda dari kebiasaan saya sebelumnya. (INN-W1)",
    "Saya menikmati situasi di mana saya bisa menggunakan dan mengembangkan kemampuan saya. (NACH-HS2)",
    "Saya percaya diri dapat mengelola aset atau sumber daya keuangan secara efektif dalam usaha, proyek, atau kegiatan saya. (SE-IF2)",
    "Saya sering mengimprovisasi metode untuk memecahkan masalah ketika jawabannya tidak jelas. (INN-CE2)",
    "Saya merasa keberhasilan dalam pekerjaan banyak ditentukan oleh faktor di luar kendali saya. (LOC-E1)",
    "Saya percaya diri dapat meyakinkan orang lain untuk memahami, mendukung, dan ikut berkomitmen pada visi serta rencana yang saya buat. (SE-M2)",
    "Saya percaya diri dapat merancang ide atau solusi baru yang sesuai dengan kebutuhan pasar. (SE-S2)",
    "Saya percaya bahwa hasil pekerjaan saya bergantung pada usaha dan cara saya bekerja. (LOC-I2)",
    "Saya merasa antusias ketika harus beradaptasi dengan situasi baru. (INN-W2)",
    "Saya percaya diri dapat menyusun rencana bisnis, termasuk memperkirakan permintaan pasar dan strategi pemasaran. (SE-P1)",
    "Saya menikmati mencoba ide-ide baru. (INN-O1)",
    "Saya percaya diri dapat memilih orang yang tepat untuk bekerja sama dan membantu mereka mengembangkan kemampuan yang dibutuhkan. (SE-IP2)",
    "Saya takut gagal dalam situasi agak sulit, ketika banyak hal bergantung pada saya. (NACH-FF1)"
    ]

In [ ]:
likert_widgets_A = []
ui_blocks = []

for idx, q in enumerate(questions_A):
    label = widgets.HTML(f"<b>{idx+1}. {q}</b>")
    scale_widget, ui_row = likert_item()
    likert_widgets_A.append(scale_widget)
    ui_blocks.append(widgets.VBox([label, ui_row]))

display(widgets.VBox(ui_blocks))

# BAGIAN B (INPUT)

In [ ]:
questions_B = [
    "Saya adalah seseorang yang cenderung malas (atau agak malas). (CON-2)",
    "Saya adalah seseorang yang memiliki imajinasi yang jelas/hidup (atau kaya) dan penuh fantasi. (OPE-3)",
    "Saya adalah seseorang yang sering khawatir. (NEU-1)",
    "Saya adalah seseorang yang mudah memaafkan (atau bisa memaafkan). (AGR-2)",
    "Saya adalah seseorang yang bisa keluar sendiri dan bersosialisasi (atau mudah bergaul). (EXT-2)",
    "Saya adalah seseorang yang bekerja secara teliti (atau menyeluruh). (CON-1)",
    "Saya adalah seseorang yang orisinil dan membawa ide-ide baru. (OPE-1)",
    "Saya adalah seseorang yang kadang-kadang agak kasar (atau sedikit tidak sopan) kepada orang lain. (AGR-1)",
    "Saya adalah seseorang yang santai dan dapat mengatasi stres dengan baik. (NEU-3)",
    "Saya adalah seseorang yang pendiam (atau tertutup). (EXT-3)",
    "Saya adalah seseorang yang menghargai pengalaman artistik. (OPE-2)",
    "Saya adalah seseorang yang penuh perhatian dan baik hati terhadap orang lain. (AGR-3)",
    "Saya adalah seseorang yang mudah gugup. (NEU-2)",
    "Saya adalah seseorang yang menyelesaikan tugas secara efektif dan efisien. (CON-3)",
    "Saya adalah seseorang yang komunikatif dan banyak bicara (atau cerewet). (EXT-1)"
    ]

In [ ]:
likert_widgets_B = []
ui_blocks = []

for idx, q in enumerate(questions_B):
    label = widgets.HTML(f"<b>{idx+1}. {q}</b>")
    scale_widget, ui_row = likert_item()
    likert_widgets_B.append(scale_widget)
    ui_blocks.append(widgets.VBox([label, ui_row]))

display(widgets.VBox(ui_blocks))

# INSERT DATABASE

In [ ]:
def save_all(btn):
    user_id = str(uuid.uuid4())
    kategori = kategori_input.value
    umur = umur_input.value
    gender = gender_input.value
    domisili = domisili_input.value
    created_at = datetime.now()

    answers_A = [w.value for w in likert_widgets_A]
    answers_B = [w.value for w in likert_widgets_B]

    # Kolom QA (DI-ESCAPE)
    qa_columns = [
        '[INN-CE1]', '[SE-M1]', '[NACH-FF2]', '[LOC-I1]',
        '[SE-P2]', '[INN-O2]', '[SE-IP1]', '[NACH-HS1]',
        '[SE-IF1]', '[LOC-E2]', '[SE-S1]', '[INN-W1]',
        '[NACH-HS2]', '[SE-IF2]', '[INN-CE2]', '[LOC-E1]',
        '[SE-M2]', '[SE-S2]', '[LOC-I2]', '[INN-W2]',
        '[SE-P1]', '[INN-O1]', '[SE-IP2]', '[NACH-FF1]'
    ]

    # Kolom QB (DI-ESCAPE)
    qb_columns = [
        '[CON-2]', '[OPE-3]', '[NEU-1]', '[AGR-2]',
        '[EXT-2]', '[CON-1]', '[OPE-1]', '[AGR-1]',
        '[NEU-3]', '[EXT-3]', '[OPE-2]', '[AGR-3]',
        '[NEU-2]', '[CON-3]', '[EXT-1]'
    ]

    cursor.execute(f"""
        INSERT INTO mst_tbl(
            user_id, created_at, kategori, umur, gender, domisili,
            {', '.join(qa_columns)},
            {', '.join(qb_columns)}
        )
        VALUES(
            ?, ?, ?, ?, ?, ?,
            {', '.join(['?' for _ in qa_columns])},
            {', '.join(['?' for _ in qb_columns])}
        )
    """, (
        user_id, created_at, kategori, umur, gender, domisili,
        *answers_A, *answers_B
    ))

    conn.commit()
    conn.close()

    print("Data berhasil disimpan!")
    print("user_id:", user_id)


In [ ]:
submit_all = widgets.Button(
    description="Simpan Semua Jawaban",
    button_style="success",
    layout=widgets.Layout(width='250px')
)
submit_all.on_click(save_all)

display(submit_all)

Button(button_style='success', description='Simpan Semua Jawaban', layout=Layout(width='250px'), style=ButtonS…

In [ ]:
# Koneksi ke database setiap mulai sistem
conn = sqlite3.connect("/content/bone_recommender.db")
cursor = conn.cursor()

# BAGIAN A (CONSTRAINT FILTERING)

In [ ]:
# 1. LOAD DATABASE
db_path = "/content/bone_recommender.db"
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM mst_tbl", conn)

In [ ]:
df

,user_id,created_at,kategori,umur,gender,domisili,INN-CE1,SE-M1,NACH-FF2,LOC-I1,SE-P2,INN-O2,SE-IP1,NACH-HS1,SE-IF1,LOC-E2,SE-S1,INN-W1,NACH-HS2,SE-IF2,INN-CE2,LOC-E1,SE-M2,SE-S2,LOC-I2,INN-W2,SE-P1,INN-O1,SE-IP2,NACH-FF1,CON-2,OPE-3,NEU-1,AGR-2,EXT-2,CON-1,OPE-1,AGR-1,NEU-3,EXT-3,OPE-2,AGR-3,NEU-2,CON-3,EXT-1
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,2025-12-08 08:54:20.558461,Pelaku UMKM,18,Laki-laki,,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,2025-12-08 08:57:07.574846,Pelaku UMKM,43,Perempuan,Kota Palembang,3,5,1,5,3,3,5,4,5,4,5,4,5,4,4,4,4,4,5,4,4,4,3,4,1,3,4,4,5,5,4,2,3,1,3,4,4,5,5
2,1e900595-616b-4b72-a377-8b9271682138,2025-12-08 09:00:28.921354,Profesional,23,Laki-laki,Kota Jakarta Pusat,3,4,5,5,3,4,4,4,4,4,4,4,5,4,5,4,5,4,5,5,5,4,5,4,1,2,2,5,2,5,5,4,1,5,1,2,5,5,5
3,5067b7f3-6483-4aea-af03-8848371e5907,2025-12-08 09:55:53.576736,Pelaku UMKM,31,Laki-laki,Kabupaten Lampung Selatan,2,3,4,4,5,5,3,2,4,4,3,5,2,5,5,3,4,5,3,4,4,5,3,4,4,4,2,2,4,3,2,4,4,5,2,3,5,2,4
4,be9238b6-b07c-4ae4-b254-90f835fcdd1f,2025-12-08 10:03:35.871759,Pelajar/Mahasiswa,20,Laki-laki,Kota Depok,5,4,4,4,3,4,4,3,3,2,4,5,4,4,4,4,5,4,4,3,2,3,4,5,5,3,5,5,3,4,2,4,3,4,4,3,5,4,2
5,d0d2e3a9-4e86-4f03-88e5-a906caee25b8,2025-12-10 13:58:03.379463,Pelajar/Mahasiswa,21,Perempuan,Kota Palu,2,2,3,4,1,2,1,3,3,4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,4,1,4,5,1,1,1,1,2,4,4,5,5,2,1
6,d2710fef-ae5e-4c41-85ab-deb4645fd739,2025-12-11 09:01:26.164036,Pelajar/Mahasiswa,20,Laki-laki,Kota Jakarta Barat,3,3,4,5,3,3,4,4,4,2,3,3,5,3,3,2,4,3,5,4,3,4,4,3,3,3,4,4,4,4,3,3,4,2,4,4,2,4,3
7,bd6c302c-be1e-4c54-95c3-f57962e32582,2025-12-12 05:55:58.689143,Pelajar/Mahasiswa,20,Perempuan,Kota Tangerang,4,4,3,4,4,4,4,5,3,4,4,4,4,4,5,4,4,4,4,4,3,5,4,4,4,5,4,4,3,4,5,3,4,3,5,4,4,4,5
8,58ce9430-163d-477f-b327-38a2d8102668,2025-12-12 06:27:19.365459,Pelajar/Mahasiswa,21,Laki-laki,Kota Tangerang Selatan,5,4,1,4,3,4,4,3,3,3,5,5,5,4,5,5,4,4,4,4,3,5,4,1,3,4,2,3,4,5,5,4,4,2,4,4,3,5,5


In [ ]:
# 2. DOMAIN MAPPING (qA) -> dictionary
domain_cols = {
    "innovativeness": [
        'INN-CE1', 'INN-O2', 'INN-W1', 'INN-CE2', 'INN-W2', 'INN-O1'
    ],
    "self_efficacy": [
        'SE-M1', 'SE-P2', 'SE-IP1', 'SE-IF1', 'SE-S1',
        'SE-IF2', 'SE-M2', 'SE-S2', 'SE-P1', 'SE-IP2'
    ],
    "need_achievement": [
        'NACH-FF2', 'NACH-HS1', 'NACH-HS2', 'NACH-FF1'
    ],
    "loc_internal": ['LOC-I1', 'LOC-I2'],
    "loc_external": ['LOC-E2', 'LOC-E1']
}

In [ ]:
# Hitung rata-rata tiap domain
for domain, cols in domain_cols.items():
    df[f"avg_{domain}"] = df[cols].mean(axis=1)

In [ ]:
# 3. KATEGORISASI DOMAIN KE HIGH/LOW LEVEL
def kategori_score(x):
    if x < 2.5:
        return "low"
    elif 2.5 <= x < 3.5:
        return "mid-low"
    elif 3.5 <= x < 4.25:
        return "mid-high"
    else:
        return "high"

def kategori_loc(internal, external):
    return "internal" if internal >= external else "external"

In [ ]:
df['cat_innovativeness'] = df['avg_innovativeness'].apply(kategori_score)
df['cat_self_efficacy'] = df['avg_self_efficacy'].apply(kategori_score)
df['cat_need_achievement'] = df['avg_need_achievement'].apply(kategori_score)
df['cat_loc'] = df.apply(lambda r: kategori_loc(r['avg_loc_internal'], r['avg_loc_external']), axis=1)

In [ ]:
df[['cat_innovativeness', 'cat_self_efficacy', 'cat_need_achievement', 'cat_loc']]

,cat_innovativeness,cat_self_efficacy,cat_need_achievement,cat_loc
0,low,low,low,internal
1,mid-high,mid-high,mid-high,internal
2,mid-high,mid-high,high,internal
3,high,mid-high,mid-low,internal
4,mid-high,mid-high,mid-high,internal
5,low,low,low,internal
6,mid-low,mid-low,mid-high,internal
7,high,mid-high,mid-high,internal
8,high,mid-high,mid-low,internal


## RULE-MAPPING AWAL SECTOR

In [ ]:
# 4. RULE MAPPING DOMAIN Q_A → SEKTOR
rules = {

    "self_efficacy": {
        "high": ["consumer oriented services", "construction", "non-high tech manufacturing", "software", "other business services, cutting-edge technology manufacturing", "high technology manufacturing"],
        "mid-high": ["skill-intensive services"],
        "mid-low": ["wholesale and retail market", "technology intensive services"],
        "low": []
    },

    "innovativeness": {
        "high": ["non-high tech manufacturing", "high technology manufacturing", "software", "technology intensive services", "cutting-edge technology manufacturing"],
        "mid-high": ["consumer oriented services"],
        "mid-low": ["construction", "wholesale and retail market", "skill-intensive services", "other business services"],
        "low": []
    },

    "need_achievement": {
        "high": ["consumer oriented services", "skill-intensive services", "software", "technology intensive services", "cutting-edge technology manufacturing", "high technology manufacturing"],
        "mid-high": ["construction", "non-high tech manufacturing", "other business services"],
        "mid-low": [],
        "low": ["wholesale and retail market"]
    },

    "loc": {
        "internal": ["consumer oriented services", "non-high tech manufacturing", "high technology manufacturing", "software", "technology intensive services", "skill-intensive services", "other business services", "cutting edge technology manufacturing"],
        "external": ["construction", "wholesale and retail market"]
    }
}

In [ ]:
# 5. FUZZY MATCHING — AMBIL SEMUA SEKTOR YANG COMPATIBLE

def rekomendasi_per_domain(row):

    # buat set kosong untuk nampung sektor
    # set dipakai agar tidak ada duplikasi
    sektor = set()

    # mengambil kategori domain dari user tersebut
    se = row["cat_self_efficacy"]
    inn = row["cat_innovativeness"]
    nach = row["cat_need_achievement"]
    loc = row["cat_loc"]

    # untuk setiap domain, masukkan sektor-sektor yang match ke dalam set
    # karena pakai set.update(), maka tidak ada duplikasi, bisa multi-source
    sektor.update(rules["self_efficacy"][se])
    sektor.update(rules["innovativeness"][inn])
    sektor.update(rules["need_achievement"][nach])
    sektor.update(rules["loc"][loc])

    # return semua sektor cocok dalam bentuk string, dipisahkan dengan koma
    # sorted() agar urut dan lebih rapi
    return ", ".join(sorted(sektor))

df["rekomendasi_sektor_qA"] = df.apply(rekomendasi_per_domain, axis=1)

In [ ]:
df[['user_id', 'cat_innovativeness', 'cat_self_efficacy', 'cat_need_achievement', 'cat_loc', 'rekomendasi_sektor_qA']]

,user_id,cat_innovativeness,cat_self_efficacy,cat_need_achievement,cat_loc,rekomendasi_sektor_qA
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,low,low,low,internal,"consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services, wholesale and retail market"
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,mid-high,mid-high,mid-high,internal,"construction, consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services"
2,1e900595-616b-4b72-a377-8b9271682138,mid-high,mid-high,high,internal,"consumer oriented services, cutting edge technology manufacturing, cutting-edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services"
3,5067b7f3-6483-4aea-af03-8848371e5907,high,mid-high,mid-low,internal,"consumer oriented services, cutting edge technology manufacturing, cutting-edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services"
4,be9238b6-b07c-4ae4-b254-90f835fcdd1f,mid-high,mid-high,mid-high,internal,"construction, consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services"
5,d0d2e3a9-4e86-4f03-88e5-a906caee25b8,low,low,low,internal,"consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services, wholesale and retail market"
6,d2710fef-ae5e-4c41-85ab-deb4645fd739,mid-low,mid-low,mid-high,internal,"construction, consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services, wholesale and retail market"
7,bd6c302c-be1e-4c54-95c3-f57962e32582,high,mid-high,mid-high,internal,"construction, consumer oriented services, cutting edge technology manufacturing, cutting-edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services"
8,58ce9430-163d-477f-b327-38a2d8102668,high,mid-high,mid-low,internal,"consumer oriented services, cutting edge technology manufacturing, cutting-edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services"


In [ ]:
# 6. KONVERSI KATEGORI → ANGKA UNTUK EUCLIDEAN

# mapping kategori ke angka
# konversi level domain menjadi angka 0-3
score_map = {"low": 0, "mid-low": 1, "mid-high": 2, "high": 3}
# loc hanya ada 2 kategori -> 0 dan 1
loc_map = {"external": 0, "internal": 1}

# MEMBUAT USER VECTOR
df["score_SE"] = df["cat_self_efficacy"].map(score_map)
df["score_INN"] = df["cat_innovativeness"].map(score_map)
df["score_NACH"] = df["cat_need_achievement"].map(score_map)
df["score_LOC"] = df["cat_loc"].map(loc_map)

In [ ]:
df[['user_id', 'cat_innovativeness', 'cat_self_efficacy', 'cat_need_achievement', 'cat_loc', 'rekomendasi_sektor_qA', 'score_SE', 'score_INN', 'score_NACH', 'score_LOC']]

,user_id,cat_innovativeness,cat_self_efficacy,cat_need_achievement,cat_loc,rekomendasi_sektor_qA,score_SE,score_INN,score_NACH,score_LOC
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,low,low,low,internal,"consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services, wholesale and retail market",0,0,0,1
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,mid-high,mid-high,mid-high,internal,"construction, consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services",2,2,2,1
2,1e900595-616b-4b72-a377-8b9271682138,mid-high,mid-high,high,internal,"consumer oriented services, cutting edge technology manufacturing, cutting-edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services",2,2,3,1
3,5067b7f3-6483-4aea-af03-8848371e5907,high,mid-high,mid-low,internal,"consumer oriented services, cutting edge technology manufacturing, cutting-edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services",2,3,1,1
4,be9238b6-b07c-4ae4-b254-90f835fcdd1f,mid-high,mid-high,mid-high,internal,"construction, consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services",2,2,2,1
5,d0d2e3a9-4e86-4f03-88e5-a906caee25b8,low,low,low,internal,"consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services, wholesale and retail market",0,0,0,1
6,d2710fef-ae5e-4c41-85ab-deb4645fd739,mid-low,mid-low,mid-high,internal,"construction, consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services, wholesale and retail market",1,1,2,1
7,bd6c302c-be1e-4c54-95c3-f57962e32582,high,mid-high,mid-high,internal,"construction, consumer oriented services, cutting edge technology manufacturing, cutting-edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services",2,3,2,1
8,58ce9430-163d-477f-b327-38a2d8102668,high,mid-high,mid-low,internal,"consumer oriented services, cutting edge technology manufacturing, cutting-edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services",2,3,1,1


## RULE-MAPPING SECTOR DETAIL

In [ ]:
# 7. SEKTOR PROTOTYPE (VERSI NUMERIC)
# vektor ini mewakili profil domain ideal untuk setiap sektor
sector_proto = {
    "consumer oriented services":           [3, 2, 3, 1],
    "construction":                        [3, 1, 2, 0],
    "wholesale and retail market":         [1, 1, 0, 0],
    "non high tech manufacturing":         [3, 3, 2, 1],
    "skill-intensive services":            [2, 1, 3, 1],
    "other business services":             [3, 1, 2, 1],
    "high technology manufacturing":       [3, 3, 3, 1],
    "software":                            [3, 3, 3, 1],
    "technology intensive services":       [1, 3, 3, 1],
    "cutting edge technology manufacturing":[3, 3, 3, 1]
}

In [ ]:
# 8. EUCLIDEAN TIE BREAKER
# setelah di fuzzy matching (hasil tidak "satu pilihan final", tapi beberapa sektor yang mirip/sesuai) menghasilkan beberapa sektor,
# kita perlu pilih yang paling cocok
# inilah gunakanya pakai tie breaker dengan Euclian Distance



def euclidean_distance(user_vec, sector_vec):
  # outputnya: semakin kecil jarak, semakin mirip profil user dengan sektor tersebut

    return np.sqrt(np.sum((np.array(user_vec) - np.array(sector_vec))**2))


def final_sector(row):
    sektor_list = row["rekomendasi_sektor_qA"].split(", ")
    # membaca isi kolom rekomendasi_sektor
    # memecah string menjadi list sektor
    # contoh:
    # "software, consumer oriented services" -> ["software", "consumer oriented services"]

    # kalau cuma 1 sektor, langsung return list berisi 1 value
    if len(sektor_list) == 1:
        return sektor_list

    # buat vektor profil user
    user_vec = [
        row["score_SE"],
        row["score_INN"],
        row["score_NACH"],li
        row["score_LOC"]
    ]

    # siapkan dict untuk menyimpan jarak
    jarak_dict = {}

    # hitung jarak untuk setiap sektor
      # baris demi baris:
      # loop semua sektor fuzzy
      # cek apakah sektor punya prototype numeric
      # hitung jaraknya
      # simpan ke dictionary
    for s in sektor_list:
        if s in sector_proto:
            dist = euclidean_distance(user_vec, sector_proto[s])
            jarak_dict[s] = dist

    # cari jarak minimum
    min_dist = min(jarak_dict.values())

    # ambil semua sektor yang jaraknya sama dengan min_dist
    sektor_terbaik = [s for s, d in jarak_dict.items() if d == min_dist]

    return sektor_terbaik


df["final_sector_qA"] = df.apply(final_sector, axis=1)

In [ ]:
def calculate_distances(row):
    sektor_list = row["rekomendasi_sektor_qA"].split(", ")
    user_vec = [
        row["score_SE"],
        row["score_INN"],
        row["score_NACH"],
        row["score_LOC"]
    ]

    jarak_dict = {}

    for s in sektor_list:
        if s in sector_proto:
            dist = euclidean_distance(user_vec, sector_proto[s])
            jarak_dict[s] = dist

    # urutkan dari jarak terkecil (paling mirip) ke terbesar
    jarak_sorted = dict(sorted(jarak_dict.items(), key=lambda x: x[1]))

    return jarak_sorted

In [ ]:
df["euclid_ranking_qA"] = df.apply(calculate_distances, axis=1)

df["top3_euclid_qA"] = df["euclid_ranking_qA"].apply(
    lambda d: [k for k, v in list(d.items())[:3]]
)

df["top5_euclid_qA"] = df["euclid_ranking_qA"].apply(
    lambda d: [k for k, v in list(d.items())[:5]]
)

In [ ]:
df[["rekomendasi_sektor_qA", "euclid_ranking_qA", "top3_euclid_qA", "top5_euclid_qA", "final_sector_qA"]]


,rekomendasi_sektor_qA,euclid_ranking_qA,top3_euclid_qA,top5_euclid_qA,final_sector_qA
0,"consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services, wholesale and retail market","{'wholesale and retail market': 1.7320508075688772, 'other business services': 3.7416573867739413, 'skill-intensive services': 3.7416573867739413, 'technology intensive services': 4.358898943540674, 'consumer oriented services': 4.69041575982343, 'cutting edge technology manufacturing': 5.196152422706632, 'high technology manufacturing': 5.196152422706632, 'software': 5.196152422706632}","[wholesale and retail market, other business services, skill-intensive services]","[wholesale and retail market, other business services, skill-intensive services, technology intensive services, consumer oriented services]",[wholesale and retail market]
1,"construction, consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services","{'consumer oriented services': 1.4142135623730951, 'other business services': 1.4142135623730951, 'skill-intensive services': 1.4142135623730951, 'construction': 1.7320508075688772, 'cutting edge technology manufacturing': 1.7320508075688772, 'high technology manufacturing': 1.7320508075688772, 'software': 1.7320508075688772, 'technology intensive services': 1.7320508075688772}","[consumer oriented services, other business services, skill-intensive services]","[consumer oriented services, other business services, skill-intensive services, construction, cutting edge technology manufacturing]","[consumer oriented services, other business services, skill-intensive services]"
2,"consumer oriented services, cutting edge technology manufacturing, cutting-edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services","{'consumer oriented services': 1.0, 'skill-intensive services': 1.0, 'cutting edge technology manufacturing': 1.4142135623730951, 'high technology manufacturing': 1.4142135623730951, 'software': 1.4142135623730951, 'technology intensive services': 1.4142135623730951, 'other business services': 1.7320508075688772}","[consumer oriented services, skill-intensive services, cutting edge technology manufacturing]","[consumer oriented services, skill-intensive services, cutting edge technology manufacturing, high technology manufacturing, software]","[consumer oriented services, skill-intensive services]"
3,"consumer oriented services, cutting edge technology manufacturing, cutting-edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services","{'cutting edge technology manufacturing': 2.23606797749979, 'high technology manufacturing': 2.23606797749979, 'software': 2.23606797749979, 'technology intensive services': 2.23606797749979, 'consumer oriented services': 2.449489742783178, 'other business services': 2.449489742783178, 'skill-intensive services': 2.8284271247461903}","[cutting edge technology manufacturing, high technology manufacturing, software]","[cutting edge technology manufacturing, high technology manufacturing, software, technology intensive services, consumer oriented services]","[cutting edge technology manufacturing, high technology manufacturing, software, technology intensive services]"
4,"construction, consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services","{'consumer oriented services': 1.4142135623730951, 'oth

## MAPPING SEKTOR -> CLUSTER

In [ ]:
cluster_mapping = {
    "Adaptive Services": [
        "wholesale and retail market",
        "consumer oriented services",
        "non high tech manufacturing",
        "other business services"
    ],

    "Dynamic Knowledge Innovators": [
        "skill-intensive services",
        "technology intensive services",
        "software",
        "high technology manufacturing"
    ],

    "Strategic Risk Navigators": [
        "technology intensive services",
        "skill-intensive services",
        "other business services"
    ],

    "Multidimensional Service Innovators": [
        "consumer oriented services",
        "skill-intensive services",
        "construction",
        "wholesale and retail market",
        "technology intensive services"
    ],

    "Focused Tech Innovators": [
        "non high tech manufacturing",
        "other business services",
        "software",
        "high technology manufacturing"
    ]
}


In [ ]:
# buat lookup cepat sektor → cluster
sector_to_cluster = {}


# beberapa sektor muncul di lebih dari 1 cluster
# setdefault(s, []) membuat list kosong untuk setiap sektor
# lalu .append(cluster) menambah cluster yang memiliki sektor tersebut
for cluster, sectors in cluster_mapping.items():
    for s in sectors:
        sector_to_cluster.setdefault(s, []).append(cluster)

In [ ]:
# hitung bobot cluster berdasarkan Top5 Euclidean
def cluster_weights_from_top5_qA(top5_list):

    # jika input bukan list atau list kosong → kembalikan dict kosong
    # ini untuk mencegah error jika data tidak valid
    if not isinstance(top5_list, list) or len(top5_list) == 0:
        return {}

    # membuat bobot berdasarkan peringkat (ranking)
    # index 0 = sektor paling mirip → bobot terbesar (5)
    # index 1 → bobot 4, dan seterusnya
    rank_weight = {0:5, 1:4, 2:3, 3:2, 4:1}

    # membuat dictionary bobot awal cluster
    # semua cluster dimulai dari nilai 0
    weights = {cluster: 0 for cluster in cluster_mapping}

    # loop sektor berdasarkan urutan ranking
    for idx, sector in enumerate(top5_list):

        # hanya proses sektor yang punya mapping ke cluster
        if sector in sector_to_cluster:
            # ambil bobot berdasarkan posisi ranking sektor tersebut
            w = rank_weight.get(idx, 0)

            # tambahkan bobot ke setiap cluster yang berisi sektor itu
            # 1 sektor bisa berhubungan ke lebih dari 1 cluster
            for cluster in sector_to_cluster[sector]:
                weights[cluster] += w

    return weights


# SIMPAN hasil bobot cluster
df["cluster_weights_top5_qA"] = df["top3_euclid_qA"].apply(cluster_weights_from_top5_qA)

In [ ]:
# menentukan cluster terbaik (highest weight)
def best_cluster_from_weights_qA(wdict):
    # jika dictionary kosong → return string kosong
    if not wdict:
        return ""
    # ambil nilai bobot tertinggi
    max_score = max(wdict.values())
    # ambil semua cluster yang memiliki bobot sama dengan bobot tertinggi
    best = [c for c, v in wdict.items() if v == max_score]
    return ", ".join(best)

df["cluster_top5_best_qA"] = df["cluster_weights_top5_qA"].apply(best_cluster_from_weights_qA)

In [ ]:
# menyimpan ranking cluster dari bobot (descending)
def sort_cluster_weights(wdict):
    return sorted(wdict.items(), key=lambda x: x[1], reverse=True)

df["cluster_top5_ranking_qA"] = df["cluster_weights_top5_qA"].apply(sort_cluster_weights)

In [ ]:
df[['user_id', 'cluster_top5_ranking_qA', 'cluster_weights_top5_qA']]

,user_id,cluster_top5_ranking_qA,cluster_weights_top5_qA
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,"[(Adaptive Services, 9), (Multidimensional Service Innovators, 8), (Strategic Risk Navigators, 7), (Focused Tech Innovators, 4), (Dynamic Knowledge Innovators, 3)]","{'Adaptive Services': 9, 'Dynamic Knowledge Innovators': 3, 'Strategic Risk Navigators': 7, 'Multidimensional Service Innovators': 8, 'Focused Tech Innovators': 4}"
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,"[(Adaptive Services, 9), (Multidimensional Service Innovators, 8), (Strategic Risk Navigators, 7), (Focused Tech Innovators, 4), (Dynamic Knowledge Innovators, 3)]","{'Adaptive Services': 9, 'Dynamic Knowledge Innovators': 3, 'Strategic Risk Navigators': 7, 'Multidimensional Service Innovators': 8, 'Focused Tech Innovators': 4}"
2,1e900595-616b-4b72-a377-8b9271682138,"[(Multidimensional Service Innovators, 9), (Adaptive Services, 5), (Dynamic Knowledge Innovators, 4), (Strategic Risk Navigators, 4), (Focused Tech Innovators, 0)]","{'Adaptive Services': 5, 'Dynamic Knowledge Innovators': 4, 'Strategic Risk Navigators': 4, 'Multidimensional Service Innovators': 9, 'Focused Tech Innovators': 0}"
3,5067b7f3-6483-4aea-af03-8848371e5907,"[(Dynamic Knowledge Innovators, 7), (Focused Tech Innovators, 7), (Adaptive Services, 0), (Strategic Risk Navigators, 0), (Multidimensional Service Innovators, 0)]","{'Adaptive Services': 0, 'Dynamic Knowledge Innovators': 7, 'Strategic Risk Navigators': 0, 'Multidimensional Service Innovators': 0, 'Focused Tech Innovators': 7}"
4,be9238b6-b07c-4ae4-b254-90f835fcdd1f,"[(Adaptive Services, 9), (Multidimensional Service Innovators, 8), (Strategic Risk Navigators, 7), (Focused Tech Innovators, 4), (Dynamic Knowledge Innovators, 3)]","{'Adaptive Services': 9, 'Dynamic Knowledge Innovators': 3, 'Strategic Risk Navigators': 7, 'Multidimensional Service Innovators': 8, 'Focused Tech Innovators': 4}"
5,d0d2e3a9-4e86-4f03-88e5-a906caee25b8,"[(Adaptive Services, 9), (Multidimensional Service Innovators, 8), (Strategic Risk Navigators, 7), (Focused Tech Innovators, 4), (Dynamic Knowledge Innovators, 3)]","{'Adaptive Services': 9, 'Dynamic Knowledge Innovators': 3, 'Strategic Risk Navigators': 7, 'Multidimensional Service Innovators': 8, 'Focused Tech Innovators': 4}"
6,d2710fef-ae5e-4c41-85ab-deb4645fd739,"[(Strategic Risk Navigators, 9), (Multidimensional Service Innovators, 8), (Dynamic Knowledge Innovators, 5), (Adaptive Services, 4), (Focused Tech Innovators, 4)]","{'Adaptive Services': 4, 'Dynamic Knowledge Innovators': 5, 'Strategic Risk Navigators': 9, 'Multidimensional Service Innovators': 8, 'Focused Tech Innovators': 4}"
7,bd6c302c-be1e-4c54-95c3-f57962e32582,"[(Dynamic Knowledge Innovators, 7), (Focused Tech Innovators, 7), (Adaptive Services, 0), (Strategic Risk Navigators, 0), (Multidimensional Service Innovators, 0)]","{'Adaptive Services': 0, 'Dynamic Knowledge Innovators': 7, 'Strategic Risk Navigators': 0, 'Multidimensional Service Innovators': 0, 'Focused Tech Innovators': 7}"
8,58ce9430-163d-477f-b327-38a2d8102668,"[(Dynamic Knowledge Innovators, 7), (Focused Tech Innovators, 7), (Adaptive Services, 0), (Strategic Risk Navigators, 0), (Multidimensional Service Innovators, 0)]","{'Adaptive Services': 0, 'Dynamic Knowledge Innovators': 7, 'Strategic Risk Navigators': 0, 'Multidimensional Service Innovators': 0, 'Focused Tech Innovators': 7}"


In [ ]:
# untuk mapping top 3 sektor ada di cluster mana aja
def top5_clusters_with_sectors_qA(top5_list):

    # jika datanya tidak valid -> kembalikan dict kosong
    if not isinstance(top5_list, list) or len(top5_list) == 0:
        return {}

    # buat dictionary cluster -> list sektor
    hasil = {cluster: [] for cluster in cluster_mapping}

    for sector in top5_list:
        if sector in sector_to_cluster:
            # cek cluster apa saja yg menaungi sektor ini
            clusters = sector_to_cluster[sector]

            # masukkan sektor ke masing-masing cluster tersebut
            for c in clusters:
                hasil[c].append(sector)

    # hapus cluster yang kosong (tidak ada sektor Top 5 di dalamnya)
    hasil = {c: sectors for c, sectors in hasil.items() if len(sectors) > 0}

    return hasil


# simpan ke dataframe
df["top5_cluster_to_sector_qA"] = df["top3_euclid_qA"].apply(top5_clusters_with_sectors_qA)

In [ ]:
def best_cluster_from_weights_qA(wdict):
    # jika kosong → return list kosong
    if not wdict:
        return []

    # cari nilai bobot tertinggi
    max_score = max(wdict.values())

    # ambil semua cluster yang memiliki bobot sama dengan max_score
    best_clusters = [c for c, v in wdict.items() if v == max_score]

    return best_clusters

In [ ]:
df["cluster_top5_best_qA"] = df["cluster_weights_top5_qA"].apply(best_cluster_from_weights_qA)

## output akhir untuk bagian A

In [ ]:
df[['user_id','top5_euclid_qA' ,  'top3_euclid_qA', 'cluster_top5_ranking_qA', 'cluster_weights_top5_qA', 'top5_cluster_to_sector_qA', 'cluster_top5_best_qA']]

,user_id,top5_euclid_qA,top3_euclid_qA,cluster_top5_ranking_qA,cluster_weights_top5_qA,top5_cluster_to_sector_qA,cluster_top5_best_qA
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,"[wholesale and retail market, other business services, skill-intensive services, technology intensive services, consumer oriented services]","[wholesale and retail market, other business services, skill-intensive services]","[(Adaptive Services, 9), (Multidimensional Service Innovators, 8), (Strategic Risk Navigators, 7), (Focused Tech Innovators, 4), (Dynamic Knowledge Innovators, 3)]","{'Adaptive Services': 9, 'Dynamic Knowledge Innovators': 3, 'Strategic Risk Navigators': 7, 'Multidimensional Service Innovators': 8, 'Focused Tech Innovators': 4}","{'Adaptive Services': ['wholesale and retail market', 'other business services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['other business services', 'skill-intensive services'], 'Multidimensional Service Innovators': ['wholesale and retail market', 'skill-intensive services'], 'Focused Tech Innovators': ['other business services']}",[Adaptive Services]
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,"[consumer oriented services, other business services, skill-intensive services, construction, cutting edge technology manufacturing]","[consumer oriented services, other business services, skill-intensive services]","[(Adaptive Services, 9), (Multidimensional Service Innovators, 8), (Strategic Risk Navigators, 7), (Focused Tech Innovators, 4), (Dynamic Knowledge Innovators, 3)]","{'Adaptive Services': 9, 'Dynamic Knowledge Innovators': 3, 'Strategic Risk Navigators': 7, 'Multidimensional Service Innovators': 8, 'Focused Tech Innovators': 4}","{'Adaptive Services': ['consumer oriented services', 'other business services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['other business services', 'skill-intensive services'], 'Multidimensional Service Innovators': ['consumer oriented services', 'skill-intensive services'], 'Focused Tech Innovators': ['other business services']}",[Adaptive Services]
2,1e900595-616b-4b72-a377-8b9271682138,"[consumer oriented services, skill-intensive services, cutting edge technology manufacturing, high technology manufacturing, software]","[consumer oriented services, skill-intensive services, cutting edge technology manufacturing]","[(Multidimensional Service Innovators, 9), (Adaptive Services, 5), (Dynamic Knowledge Innovators, 4), (Strategic Risk Navigators, 4), (Focused Tech Innovators, 0)]","{'Adaptive Services': 5, 'Dynamic Knowledge Innovators': 4, 'Strategic Risk Navigators': 4, 'Multidimensional Service Innovators': 9, 'Focused Tech Innovators': 0}","{'Adaptive Services': ['consumer oriented services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['skill-intensive services'], 'Multidimensional Service Innovators': ['consumer oriented services', 'skill-intensive services']}",[Multidimensional Service Innovators]
3,5067b7f3-6483-4aea-af03-8848371e5907,"[cutting edge technology manufacturing, high technology manufacturing, software, technology intensive services, consumer oriented services]","[cutting edge technology manufacturing, high technology manufacturing, software]","[(Dynamic Knowledge Innovators, 7), (Focused Tech Innovators, 7), (Adaptive Services, 0), (Strategic Risk Navigators, 0), (Multidimensional Service Innovators, 0)]","{'Adaptive Services': 0, 'Dynamic Knowledge Innovators': 7, 'Strategic Risk Navigators': 0, 'Multidimensional Service Innovators': 0, 'Focused Tech Innovators': 7}","{'Dynamic Knowledge Innovators': ['high technology manufacturing', 'software'], 'Focused Tech Innovators': ['high technology manufacturing', 'software']}","[Dynamic Knowledge Innovators, Focused Tech Innovators]"
4,be9238b6-b07c-4ae4-b254-90f835fcdd1f,"[consumer oriented services, other business services, skill-intensive services, construction, cuttin

# BAGIAN B (CONSTRAINT FILTERING)

In [ ]:
# dictionary untuk narik data dari database
big5 = {
    "openess": ['OPE-3', 'OPE-1', 'OPE-2'],

    "conscientiousness": {
        "normal": ['CON-1', 'CON-3'],
        "reverse": ['CON-2']
    },

    "extraversion": {
        "normal": ['EXT-2', 'EXT-1'],
        "reverse": ['EXT-3']
    },

    "agreeableness": {
        "normal": ['AGR-2', 'AGR-3'],
        "reverse": ['AGR-1']
    },

    "neuroticism": {
        "normal": ['NEU-1', 'NEU-2'],
        "reverse": ['NEU-3']
    }
}

In [ ]:
def reverse_score(x):
    if pd.isna(x):
        return x
    return 6 - x   # reverse dari skala 1-5

## AVERAGE

Openess

In [ ]:
df["avg_openess"] = df[big5["openess"]].mean(axis=1)

Conscientiousness

In [ ]:
df["con_scored"] = (
    df[big5["conscientiousness"]["normal"]].mean(axis=1) +
    df[big5["conscientiousness"]["reverse"]].applymap(reverse_score).mean(axis=1)
) / 2

df["avg_conscientiousness"] = df["con_scored"]
df.drop(columns=["con_scored"], inplace=True)

/tmp/ipython-input-2136505909.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[big5["conscientiousness"]["reverse"]].applymap(reverse_score).mean(axis=1)


Extraversion

In [ ]:
df["ext_scored"] = (
    df[big5["extraversion"]["normal"]].mean(axis=1) +
    df[big5["extraversion"]["reverse"]].applymap(reverse_score).mean(axis=1)
) / 2

df["avg_extraversion"] = df["ext_scored"]
df.drop(columns=["ext_scored"], inplace=True)

/tmp/ipython-input-1504917528.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[big5["extraversion"]["reverse"]].applymap(reverse_score).mean(axis=1)


Agreeableness

In [ ]:
df["agr_scored"] = (
    df[big5["agreeableness"]["normal"]].mean(axis=1) +
    df[big5["agreeableness"]["reverse"]].applymap(reverse_score).mean(axis=1)
) / 2

df["avg_agreeableness"] = df["agr_scored"]
df.drop(columns=["agr_scored"], inplace=True)

/tmp/ipython-input-3368654431.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[big5["agreeableness"]["reverse"]].applymap(reverse_score).mean(axis=1)


Neuroticism

In [ ]:
df["neu_scored"] = (
    df[big5["neuroticism"]["normal"]].mean(axis=1) +
    df[big5["neuroticism"]["reverse"]].applymap(reverse_score).mean(axis=1)
) / 2

df["avg_neuroticism"] = df["neu_scored"]
df.drop(columns=["neu_scored"], inplace=True)

/tmp/ipython-input-851470318.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[big5["neuroticism"]["reverse"]].applymap(reverse_score).mean(axis=1)


In [ ]:
df[["user_id",
    "avg_openess",
    "avg_conscientiousness",
    "avg_extraversion",
    "avg_agreeableness",
    "avg_neuroticism"
]]

,user_id,avg_openess,avg_conscientiousness,avg_extraversion,avg_agreeableness,avg_neuroticism
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,1.000000,3.00,3.00,3.00,3.00
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,3.333333,5.00,5.00,4.00,3.50
2,1e900595-616b-4b72-a377-8b9271682138,2.666667,5.00,2.25,2.75,4.25
3,5067b7f3-6483-4aea-af03-8848371e5907,2.666667,2.25,2.50,2.25,2.75
4,be9238b6-b07c-4ae4-b254-90f835fcdd1f,3.000000,2.50,2.25,3.00,4.00
5,d0d2e3a9-4e86-4f03-88e5-a906caee25b8,2.000000,1.75,1.50,5.00,4.25
6,d2710fef-ae5e-4c41-85ab-deb4645fd739,3.333333,3.50,3.75,3.50,2.50
7,bd6c302c-be1e-4c54-95c3-f57962e32582,5.000000,3.00,3.50,3.50,3.00
8,58ce9430-163d-477f-b327-38a2d8102668,4.333333,4.00,4.25,2.75,2.25


## RULE-MAPPING

In [ ]:
def avg_to_notation(x):
    if np.isnan(x):
        return np.nan
    if x >= 4.2:    return '++'
    elif x >= 3.5:  return '+'
    elif x >= 2.5:  return '0'
    elif x >= 1.8:  return '-'
    else:           return '--'

In [ ]:
# buat kolom notasi O, C, E, A, N berdasarkan AVERAGE
df['Note_O'] = df['avg_openess'].apply(avg_to_notation)
df['Note_C'] = df['avg_conscientiousness'].apply(avg_to_notation)
df['Note_E'] = df['avg_extraversion'].apply(avg_to_notation)
df['Note_A'] = df['avg_agreeableness'].apply(avg_to_notation)
df['Note_N'] = df['avg_neuroticism'].apply(avg_to_notation)

In [ ]:
df

,user_id,created_at,kategori,umur,gender,domisili,INN-CE1,SE-M1,NACH-FF2,LOC-I1,SE-P2,INN-O2,SE-IP1,NACH-HS1,SE-IF1,LOC-E2,SE-S1,INN-W1,NACH-HS2,SE-IF2,INN-CE2,LOC-E1,SE-M2,SE-S2,LOC-I2,INN-W2,SE-P1,INN-O1,SE-IP2,NACH-FF1,CON-2,OPE-3,NEU-1,AGR-2,EXT-2,CON-1,OPE-1,AGR-1,NEU-3,EXT-3,OPE-2,AGR-3,NEU-2,CON-3,EXT-1,avg_innovativeness,avg_self_efficacy,avg_need_achievement,avg_loc_internal,avg_loc_external,cat_innovativeness,cat_self_efficacy,cat_need_achievement,cat_loc,rekomendasi_sektor_qA,score_SE,score_INN,score_NACH,score_LOC,final_sector_qA,euclid_ranking_qA,top3_euclid_qA,top5_euclid_qA,cluster_weights_top5_qA,cluster_top5_best_qA,cluster_top5_ranking_qA,top5_cluster_to_sector_qA,avg_openess,avg_conscientiousness,avg_extraversion,avg_agreeableness,avg_neuroticism,Note_O,Note_C,Note_E,Note_A,Note_N
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,2025-12-08 08:54:20.558461,Pelaku UMKM,18,Laki-laki,,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.000000,1.0,1.0,1.0,1.0,low,low,low,internal,"consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services, wholesale and retail market",0,0,0,1,[wholesale and retail market],"{'wholesale and retail market': 1.7320508075688772, 'other business services': 3.7416573867739413, 'skill-intensive services': 3.7416573867739413, 'technology intensive services': 4.358898943540674, 'consumer oriented services': 4.69041575982343, 'cutting edge technology manufacturing': 5.196152422706632, 'high technology manufacturing': 5.196152422706632, 'software': 5.196152422706632}","[wholesale and retail market, other business services, skill-intensive services]","[wholesale and retail market, other business services, skill-intensive services, technology intensive services, consumer oriented services]","{'Adaptive Services': 9, 'Dynamic Knowledge Innovators': 3, 'Strategic Risk Navigators': 7, 'Multidimensional Service Innovators': 8, 'Focused Tech Innovators': 4}",[Adaptive Services],"[(Adaptive Services, 9), (Multidimensional Service Innovators, 8), (Strategic Risk Navigators, 7), (Focused Tech Innovators, 4), (Dynamic Knowledge Innovators, 3)]","{'Adaptive Services': ['wholesale and retail market', 'other business services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['other business services', 'skill-intensive services'], 'Multidimensional Service Innovators': ['wholesale and retail market', 'skill-intensive services'], 'Focused Tech Innovators': ['other business services']}",1.000000,3.00,3.00,3.00,3.00,--,0,0,0,0
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,2025-12-08 08:57:07.574846,Pelaku UMKM,43,Perempuan,Kota Palembang,3,5,1,5,3,3,5,4,5,4,5,4,5,4,4,4,4,4,5,4,4,4,3,4,1,3,4,4,5,5,4,2,3,1,3,4,4,5,5,3.666667,4.2,3.5,5.0,4.0,mid-high,mid-high,mid-high,internal,"construction, consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services",2,2,2,1,"[consumer oriented services, other business services, skill-intensive services]","{'consumer oriented services': 1.4142135623730951, 'other business services': 1.4142135623730951, 'skill-intensive services': 1.4142135623730951, 'construction': 1.7320508075688772, 'cutting edge technology manufacturing': 1.7320508075688772, 'high technology manufacturing': 1.7320508075688772, 'software': 1.7320508075688772, 'technology intensive services': 1.7320508075688772}","[consumer oriented services, other business services, skill-intensive services]","[consumer oriented services, other business services, skill-intensive services, construction, cutting edge technology manufacturing]","{'Adaptive Services': 9, 'Dynamic Knowledge Innovators': 3, 'Strategic Risk Navigators': 7, 'Multidimensional Service I

In [ ]:
# definisikan rule cluster
clusters = {
    'Adaptive Services': {
        'O':'++', 'C':'0/+', 'E':'+',  'A':'0',    'N':'--'
    },
    'Dynamic Knowledge Innovators': {
        'O':'++', 'C':'-',   'E':'++', 'A':'-',    'N':'--'
    },
    'Strategic Risk Navigators': {
        'O':'++', 'C':'-',   'E':'+',  'A':'-',    'N':'--'
    },
    'Multidimensional Service Innovators': {
        'O':'++','C':'-/+','E':'0/++', 'A':'--/0','N':'--/+'
    },
    'Focused Tech Innovators': {
        'O':'++','C':'-','E':'-/+','A':'0/+','N':'-'
    }
    # ,
    # 'Gender-Related Traits': {
    #     'O':'--/++', 'C':'--/++', 'E':'--/++', 'A':'--/++', 'N':'+/++'
    # }
}

In [ ]:
# fungsi cek kecocokan simbol (mendukung “0/+”, “-/+”, “--/+”)
# intinya cek apakah simbol user cocok dengan simbol cluster
def symbol_matches(resp_sym, cluster_sym):
    if pd.isna(resp_sym):
        return False                # kalau user tidak punya nilai → dianggap tidak cocok
    options = cluster_sym.split('/') # contoh "0/+" → ["0", "+"]
    return resp_sym in options       # cocok kalau simbol user ada di daftar opsi

In [ ]:
# mapping simbol ke angka untuk tie-breaker

symbol_to_num = {'++': 2.0, '+': 1.0, '0': 0.0, '-': -1.0, '--': -2.0} # supaya simbol bisa dipakai dalam perhitungan jarak

# mengubah rule cluster (notasi) -> vektor angka
def cluster_proto_numeric(cluster_sig):
    vals = []  # simpan angka O, C, E, A, N
    for dim in ['O','C','E','A','N']:   # per dimensi
        opts = cluster_sig[dim].split('/')  # contoh "0/+" → ["0","+"]
        nums = [symbol_to_num[o] for o in opts]  # ubah simbol ke angka
        vals.append(np.mean(nums))  # ambil nilai rata-rata
    return np.array(vals)           # kembalikan sebagai vektor numpy

# menghitung prototype vector untuk semua cluster
cluster_protos = {name: cluster_proto_numeric(sig) for name, sig in clusters.items()}

In [ ]:
# # Fungsi assign cluster berbasis matching score + tie breaker
# def assign_cluster_from_avg(row):

#     best_score = -1
#     best_clusters = []

#     # tahap 1 → hitung match per cluster
#     for name, sig in clusters.items():
#         matches = 0
#         for dim in ['O','C','E','A','N']:
#             if symbol_matches(row[f'Note_{dim}'], sig[dim]):
#                 matches += 1
#         score = matches / 5.0

#         if score > best_score:
#             best_score = score
#             best_clusters = [name]
#         elif score == best_score:
#             best_clusters.append(name)

#     # jika hanya satu cluster yang menang
#     if len(best_clusters) == 1:
#         return best_clusters[0], best_score

#     # tahap 2 → tie-breaker berdasarkan jarak euclidean
#     user_vec = np.array([
#         symbol_to_num[row['Note_O']],
#         symbol_to_num[row['Note_C']],
#         symbol_to_num[row['Note_E']],
#         symbol_to_num[row['Note_A']],
#         symbol_to_num[row['Note_N']]
#     ])

#     best_dist = np.inf
#     best_name = None

#     for name in best_clusters:
#         dist = np.linalg.norm(user_vec - cluster_protos[name])
#         if dist < best_dist:
#             best_dist = dist
#             best_name = name

#     return best_name, best_score

In [ ]:
# assignments = df.apply(assign_cluster_from_avg, axis=1)

# df['best_cluster_qB'] = assignments.apply(lambda x: x[0])
# df['best_cluster_qB_Score'] = assignments.apply(lambda x: x[1])

In [ ]:
# df[["user_id", "cluster_top5_best_qA", "best_cluster_qB"]]

In [ ]:
# TOP-5 CLUSTER RANKING FOR QB

# Fungsi: hitung matching score semua cluster (0–1)
def compute_cluster_scores(row):
    cluster_scores = {}

    for name, sig in clusters.items():
        matches = 0
        for dim in ['O','C','E','A','N']:
            if symbol_matches(row[f"Note_{dim}"], sig[dim]):
                matches += 1

        cluster_scores[name] = matches / 5.0

    return cluster_scores


# Fungsi utama dengan tie-breaker + menyimpan top-5 ranking
def assign_cluster_with_top5(row):

    # 1. Hitung matching score semua cluster
    scores = compute_cluster_scores(row)

    # 2. Ranking cluster berdasarkan score (descending)
    sorted_clusters = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # Ambil top-5 cluster
    top5_scores = sorted_clusters[:5]
    top5_names = [x[0] for x in top5_scores]

    # 3. Cari cluster pemenang — cek apakah ada tie
    best_score_value = top5_scores[0][1]
    tied_clusters = [c for c, s in top5_scores if s == best_score_value]

    # Jika hanya satu cluster → langsung pemenang
    if len(tied_clusters) == 1:
        best_cluster = tied_clusters[0]
        return best_cluster, best_score_value, top5_names, top5_scores

    # 4. Tie-breaker → Euclidean Distance
    user_vec = np.array([
        symbol_to_num[row['Note_O']],
        symbol_to_num[row['Note_C']],
        symbol_to_num[row['Note_E']],
        symbol_to_num[row['Note_A']],
        symbol_to_num[row['Note_N']]
    ])

    best_dist = np.inf
    best_name = None

    for name in tied_clusters:
        dist = np.linalg.norm(user_vec - cluster_protos[name])
        if dist < best_dist:
            best_dist = dist
            best_name = name

    return best_name, best_score_value, top5_names, top5_scores

In [ ]:
# JALANKAN KE DATAFRAME

results = df.apply(assign_cluster_with_top5, axis=1)

df["top5_clusters_qB"] = results.apply(lambda x: x[2])        # hanya nama cluster
df["top5_clusters_qB_score"] = results.apply(lambda x: x[3])  # tuple (cluster, score)

In [ ]:
def pick_best_clusters_qB_multi(row):
    top5 = row["top5_clusters_qB_score"]   # list: [(cluster, score), ...]

    if not isinstance(top5, list) or len(top5) == 0:
        return ""

    # Ambil nilai tertinggi
    highest_score = max(score for _, score in top5)

    # Jika highest score = 0, kosongkan
    if highest_score == 0:
        return ""

    # Ambil semua cluster yang skor-nya sama dengan nilai tertinggi
    best_clusters = [name for name, score in top5 if score == highest_score]

    # Gabungkan dalam 1 string
    return ", ".join(best_clusters)


def pick_best_clusters_qB_score(row):
    top5 = row["top5_clusters_qB_score"]

    if not isinstance(top5, list) or len(top5) == 0:
        return np.nan

    highest_score = max(score for _, score in top5)

    # Jika highest score = 0, return NaN
    if highest_score == 0:
        return np.nan

    return highest_score

In [ ]:
# SIMPAN KE DATAFRAME
df["cluster_top5_best_qB"] = df.apply(pick_best_clusters_qB_multi, axis=1)
df["cluster_top5_best_qB_score"] = df.apply(pick_best_clusters_qB_score, axis=1)

In [ ]:
df[[
    "user_id",
    "top5_clusters_qB",
    "top5_clusters_qB_score",
    "cluster_top5_best_qB",
    "cluster_top5_best_qB_score"
]]

,user_id,top5_clusters_qB,top5_clusters_qB_score,cluster_top5_best_qB,cluster_top5_best_qB_score
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,"[Adaptive Services, Multidimensional Service Innovators, Focused Tech Innovators, Dynamic Knowledge Innovators, Strategic Risk Navigators]","[(Adaptive Services, 0.4), (Multidimensional Service Innovators, 0.4), (Focused Tech Innovators, 0.2), (Dynamic Knowledge Innovators, 0.0), (Strategic Risk Navigators, 0.0)]","Adaptive Services, Multidimensional Service Innovators",0.4
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,"[Multidimensional Service Innovators, Dynamic Knowledge Innovators, Focused Tech Innovators, Adaptive Services, Strategic Risk Navigators]","[(Multidimensional Service Innovators, 0.4), (Dynamic Knowledge Innovators, 0.2), (Focused Tech Innovators, 0.2), (Adaptive Services, 0.0), (Strategic Risk Navigators, 0.0)]",Multidimensional Service Innovators,0.4
2,1e900595-616b-4b72-a377-8b9271682138,"[Focused Tech Innovators, Adaptive Services, Multidimensional Service Innovators, Dynamic Knowledge Innovators, Strategic Risk Navigators]","[(Focused Tech Innovators, 0.4), (Adaptive Services, 0.2), (Multidimensional Service Innovators, 0.2), (Dynamic Knowledge Innovators, 0.0), (Strategic Risk Navigators, 0.0)]",Focused Tech Innovators,0.4
3,5067b7f3-6483-4aea-af03-8848371e5907,"[Dynamic Knowledge Innovators, Strategic Risk Navigators, Multidimensional Service Innovators, Focused Tech Innovators, Adaptive Services]","[(Dynamic Knowledge Innovators, 0.4), (Strategic Risk Navigators, 0.4), (Multidimensional Service Innovators, 0.4), (Focused Tech Innovators, 0.2), (Adaptive Services, 0.0)]","Dynamic Knowledge Innovators, Strategic Risk Navigators, Multidimensional Service Innovators",0.4
4,be9238b6-b07c-4ae4-b254-90f835fcdd1f,"[Adaptive Services, Multidimensional Service Innovators, Focused Tech Innovators, Dynamic Knowledge Innovators, Strategic Risk Navigators]","[(Adaptive Services, 0.4), (Multidimensional Service Innovators, 0.4), (Focused Tech Innovators, 0.4), (Dynamic Knowledge Innovators, 0.0), (Strategic Risk Navigators, 0.0)]","Adaptive Services, Multidimensional Service Innovators, Focused Tech Innovators",0.4
5,d0d2e3a9-4e86-4f03-88e5-a906caee25b8,"[Adaptive Services, Dynamic Knowledge Innovators, Strategic Risk Navigators, Multidimensional Service Innovators, Focused Tech Innovators]","[(Adaptive Services, 0.0), (Dynamic Knowledge Innovators, 0.0), (Strategic Risk Navigators, 0.0), (Multidimensional Service Innovators, 0.0), (Focused Tech Innovators, 0.0)]",,NaN
6,d2710fef-ae5e-4c41-85ab-deb4645fd739,"[Adaptive Services, Focused Tech Innovators, Strategic Risk Navigators, Multidimensional Service Innovators, Dynamic Knowledge Innovators]","[(Adaptive Services, 0.4), (Focused Tech Innovators, 0.4), (Strategic Risk Navigators, 0.2), (Multidimensional Service Innovators, 0.2), (Dynamic Knowledge Innovators, 0.0)]","Adaptive Services, Focused Tech Innovators",0.4
7,bd6c302c-be1e-4c54-95c3-f57962e32582,"[Adaptive Services, Focused Tech Innovators, Strategic Risk Navigators, Dynamic Knowledge Innovators, Multidimensional Service Innovators]","[(Adaptive Services, 0.6), (Focused Tech Innovators, 0.6), (Strategic Risk Navigators, 0.4), (Dynamic Knowledge Innovators, 0.2), (Multidimensional Service Innovators, 0.2)]","Adaptive Services, Focused Tech Innovators",0.6
8,58ce9430-163d-477f-b327-38a2d8102668,"[Multidimensional Service Innovators, Adaptive Services, Focused Tech Innovators, Dynamic Knowledge Innovators, Strategic Risk Navigators]","[(Multidimensional Service Innovators, 0.8), (Adaptive Services, 0.6), (Focused Tech Innovators, 0.6), (Dynamic Knowledge Innovators, 0.4), (Strategic Risk Navigators, 0.2)]",Multidimensional Service Innovators,0.8


CLUSTER -> BEST SECTOR

In [ ]:
def top5_clusters_with_matched_sectors_qB(row):

    top5_clusters_qB = row["top5_clusters_qB"]          # list cluster nama
    top3_sectors_qA  = row["top3_euclid_qA"]            # list sektor Top-3 Euclid qA

    # validasi
    if not isinstance(top5_clusters_qB, list) or not isinstance(top3_sectors_qA, list):
        return {}

    hasil = {}

    for cluster_name in top5_clusters_qB:

        # pastikan cluster ada di mapping
        if cluster_name in cluster_mapping:

            # ambil semua sektor cluster tsb
            sektor_cluster = cluster_mapping[cluster_name]

            # cocokkan dgn top3 euclid qA
            sektor_match = [s for s in sektor_cluster if s in top3_sectors_qA]

            # jika ada kecocokan → simpan
            if len(sektor_match) > 0:
                hasil[cluster_name] = sektor_match

    return hasil

# simpan hasil ke dataframe
df["top5_cluster_to_sector_qB"] = df.apply(top5_clusters_with_matched_sectors_qB, axis=1)

## OUTPUT UNTUK MEMBANDINGKAN A VS B

In [ ]:
df[["user_id",
    "top3_euclid_qA",
    "cluster_top5_best_qA",
    "cluster_top5_ranking_qA",
    "top5_cluster_to_sector_qA",
    "cluster_top5_best_qB",
    "top5_clusters_qB_score",
    "top5_cluster_to_sector_qB"
    ]]

,user_id,top3_euclid_qA,cluster_top5_best_qA,cluster_top5_ranking_qA,top5_cluster_to_sector_qA,cluster_top5_best_qB,top5_clusters_qB_score,top5_cluster_to_sector_qB
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,"[wholesale and retail market, other business services, skill-intensive services]",[Adaptive Services],"[(Adaptive Services, 9), (Multidimensional Service Innovators, 8), (Strategic Risk Navigators, 7), (Focused Tech Innovators, 4), (Dynamic Knowledge Innovators, 3)]","{'Adaptive Services': ['wholesale and retail market', 'other business services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['other business services', 'skill-intensive services'], 'Multidimensional Service Innovators': ['wholesale and retail market', 'skill-intensive services'], 'Focused Tech Innovators': ['other business services']}","Adaptive Services, Multidimensional Service Innovators","[(Adaptive Services, 0.4), (Multidimensional Service Innovators, 0.4), (Focused Tech Innovators, 0.2), (Dynamic Knowledge Innovators, 0.0), (Strategic Risk Navigators, 0.0)]","{'Adaptive Services': ['wholesale and retail market', 'other business services'], 'Multidimensional Service Innovators': ['skill-intensive services', 'wholesale and retail market'], 'Focused Tech Innovators': ['other business services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['skill-intensive services', 'other business services']}"
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,"[consumer oriented services, other business services, skill-intensive services]",[Adaptive Services],"[(Adaptive Services, 9), (Multidimensional Service Innovators, 8), (Strategic Risk Navigators, 7), (Focused Tech Innovators, 4), (Dynamic Knowledge Innovators, 3)]","{'Adaptive Services': ['consumer oriented services', 'other business services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['other business services', 'skill-intensive services'], 'Multidimensional Service Innovators': ['consumer oriented services', 'skill-intensive services'], 'Focused Tech Innovators': ['other business services']}",Multidimensional Service Innovators,"[(Multidimensional Service Innovators, 0.4), (Dynamic Knowledge Innovators, 0.2), (Focused Tech Innovators, 0.2), (Adaptive Services, 0.0), (Strategic Risk Navigators, 0.0)]","{'Multidimensional Service Innovators': ['consumer oriented services', 'skill-intensive services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Focused Tech Innovators': ['other business services'], 'Adaptive Services': ['consumer oriented services', 'other business services'], 'Strategic Risk Navigators': ['skill-intensive services', 'other business services']}"
2,1e900595-616b-4b72-a377-8b9271682138,"[consumer oriented services, skill-intensive services, cutting edge technology manufacturing]",[Multidimensional Service Innovators],"[(Multidimensional Service Innovators, 9), (Adaptive Services, 5), (Dynamic Knowledge Innovators, 4), (Strategic Risk Navigators, 4), (Focused Tech Innovators, 0)]","{'Adaptive Services': ['consumer oriented services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['skill-intensive services'], 'Multidimensional Service Innovators': ['consumer oriented services', 'skill-intensive services']}",Focused Tech Innovators,"[(Focused Tech Innovators, 0.4), (Adaptive Services, 0.2), (Multidimensional Service Innovators, 0.2), (Dynamic Knowledge Innovators, 0.0), (Strategic Risk Navigators, 0.0)]","{'Adaptive Services': ['consumer oriented services'], 'Multidimensional Service Innovators': ['consumer oriented services', 'skill-intensive services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['skill-intensive services']}"
3,5067b7f3-6483-4aea-af03-8848371e5907,"[cutting edge technology manufacturing, high technology manufacturing, software]","[Dynamic Know

In [ ]:
df

,user_id,created_at,kategori,umur,gender,domisili,INN-CE1,SE-M1,NACH-FF2,LOC-I1,SE-P2,INN-O2,SE-IP1,NACH-HS1,SE-IF1,LOC-E2,SE-S1,INN-W1,NACH-HS2,SE-IF2,INN-CE2,LOC-E1,SE-M2,SE-S2,LOC-I2,INN-W2,SE-P1,INN-O1,SE-IP2,NACH-FF1,CON-2,OPE-3,NEU-1,AGR-2,EXT-2,CON-1,OPE-1,AGR-1,NEU-3,EXT-3,OPE-2,AGR-3,NEU-2,CON-3,EXT-1,avg_innovativeness,avg_self_efficacy,avg_need_achievement,avg_loc_internal,avg_loc_external,cat_innovativeness,cat_self_efficacy,cat_need_achievement,cat_loc,rekomendasi_sektor_qA,score_SE,score_INN,score_NACH,score_LOC,final_sector_qA,euclid_ranking_qA,top3_euclid_qA,top5_euclid_qA,cluster_weights_top5_qA,cluster_top5_best_qA,cluster_top5_ranking_qA,top5_cluster_to_sector_qA,avg_openess,avg_conscientiousness,avg_extraversion,avg_agreeableness,avg_neuroticism,Note_O,Note_C,Note_E,Note_A,Note_N,top5_clusters_qB,top5_clusters_qB_score,cluster_top5_best_qB,cluster_top5_best_qB_score,top5_cluster_to_sector_qB
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,2025-12-08 08:54:20.558461,Pelaku UMKM,18,Laki-laki,,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.000000,1.0,1.0,1.0,1.0,low,low,low,internal,"consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business services, skill-intensive services, software, technology intensive services, wholesale and retail market",0,0,0,1,[wholesale and retail market],"{'wholesale and retail market': 1.7320508075688772, 'other business services': 3.7416573867739413, 'skill-intensive services': 3.7416573867739413, 'technology intensive services': 4.358898943540674, 'consumer oriented services': 4.69041575982343, 'cutting edge technology manufacturing': 5.196152422706632, 'high technology manufacturing': 5.196152422706632, 'software': 5.196152422706632}","[wholesale and retail market, other business services, skill-intensive services]","[wholesale and retail market, other business services, skill-intensive services, technology intensive services, consumer oriented services]","{'Adaptive Services': 9, 'Dynamic Knowledge Innovators': 3, 'Strategic Risk Navigators': 7, 'Multidimensional Service Innovators': 8, 'Focused Tech Innovators': 4}",[Adaptive Services],"[(Adaptive Services, 9), (Multidimensional Service Innovators, 8), (Strategic Risk Navigators, 7), (Focused Tech Innovators, 4), (Dynamic Knowledge Innovators, 3)]","{'Adaptive Services': ['wholesale and retail market', 'other business services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['other business services', 'skill-intensive services'], 'Multidimensional Service Innovators': ['wholesale and retail market', 'skill-intensive services'], 'Focused Tech Innovators': ['other business services']}",1.000000,3.00,3.00,3.00,3.00,--,0,0,0,0,"[Adaptive Services, Multidimensional Service Innovators, Focused Tech Innovators, Dynamic Knowledge Innovators, Strategic Risk Navigators]","[(Adaptive Services, 0.4), (Multidimensional Service Innovators, 0.4), (Focused Tech Innovators, 0.2), (Dynamic Knowledge Innovators, 0.0), (Strategic Risk Navigators, 0.0)]","Adaptive Services, Multidimensional Service Innovators",0.4,"{'Adaptive Services': ['wholesale and retail market', 'other business services'], 'Multidimensional Service Innovators': ['skill-intensive services', 'wholesale and retail market'], 'Focused Tech Innovators': ['other business services'], 'Dynamic Knowledge Innovators': ['skill-intensive services'], 'Strategic Risk Navigators': ['skill-intensive services', 'other business services']}"
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,2025-12-08 08:57:07.574846,Pelaku UMKM,43,Perempuan,Kota Palembang,3,5,1,5,3,3,5,4,5,4,5,4,5,4,4,4,4,4,5,4,4,4,3,4,1,3,4,4,5,5,4,2,3,1,3,4,4,5,5,3.666667,4.2,3.5,5.0,4.0,mid-high,mid-high,mid-high,internal,"construction, consumer oriented services, cutting edge technology manufacturing, high technology manufacturing, non-high tech manufacturing, other business service

# COMPARISSON RECOMMENDATION BAGIAN A VS B

In [ ]:
def get_recommended_cluster_refined(row):

    qA = row.get("cluster_top5_best_qA", [])
    qB = row.get("cluster_top5_best_qB", [])
    mapping = row.get("top5_cluster_to_sector_qA", {}) or {}

    # Normalisasi input
    if isinstance(qA, str):
        qA = [x.strip() for x in qA.split(",")] if qA.strip() else []
    if isinstance(qB, str):
        qB = [x.strip() for x in qB.split(",")] if qB.strip() else []

    # Jika salah satu kosong
    if not qA or not qB:
        return ["Belum dapat menentukan cluster yang sesuai."]

    # RULE 1 — Irisan
    intersection = [c for c in qA if c in qB]
    if intersection:
        initial = intersection[:]
    else:
        # RULE 2 — Ambil terbaik masing-masing
        initial = [qA[0], qB[0]]

    # Validasi kandidat awal
    valid_initial = [
        c for c in initial
        if c in mapping and len(mapping.get(c, [])) > 0
    ]

    final_candidates = valid_initial.copy()

    # Semua kandidat yang tersedia
    all_clusters = list(dict.fromkeys(qA + qB))

    def sektor_baru(cluster, sektor_terwakili):
        return set(mapping.get(cluster, [])) - sektor_terwakili

    # Substitusi jika ada kandidat invalid
    for invalid in set(initial) - set(valid_initial):

        sektor_terwakili = set()
        for c in final_candidates:
            sektor_terwakili.update(mapping.get(c, []))

        best_choice = None
        best_gain = -1

        for candidate in all_clusters:
            if candidate in final_candidates:
                continue
            if candidate not in mapping:
                continue

            gain = len(sektor_baru(candidate, sektor_terwakili))
            if gain > best_gain:
                best_gain = gain
                best_choice = candidate

        if best_choice:
            final_candidates.append(best_choice)

    # RULE tambahan — cover sektor user
    sektor_user = set(row.get("top3_euclid_qA", []))

    sektor_terwakili = set()
    for c in final_candidates:
        sektor_terwakili.update(mapping.get(c, []))

    sektor_kurang = sektor_user - sektor_terwakili

    if sektor_kurang:
        best_choice = None
        best_gain = -1

        for candidate in all_clusters:
            if candidate in final_candidates:
                continue
            if candidate not in mapping:
                continue

            gain = len(set(mapping[candidate]) & sektor_kurang)
            if gain > best_gain:
                best_gain = gain
                best_choice = candidate

        if best_choice:
            final_candidates.append(best_choice)

    return final_candidates


In [ ]:
df["recommended_cluster"] = df.apply(
    get_recommended_cluster_refined,
    axis=1
)

In [ ]:
df[["user_id", "recommended_cluster", "top3_euclid_qA"]]

,user_id,recommended_cluster,top3_euclid_qA
0,9fc7ede8-fabc-4edd-8900-7dfdf3368e87,"[Adaptive Services, Multidimensional Service Innovators]","[wholesale and retail market, other business services, skill-intensive services]"
1,80e930fe-2bd5-4c10-bf74-abe964d92ebf,"[Adaptive Services, Multidimensional Service Innovators]","[consumer oriented services, other business services, skill-intensive services]"
2,1e900595-616b-4b72-a377-8b9271682138,[Multidimensional Service Innovators],"[consumer oriented services, skill-intensive services, cutting edge technology manufacturing]"
3,5067b7f3-6483-4aea-af03-8848371e5907,"[Dynamic Knowledge Innovators, Focused Tech Innovators]","[cutting edge technology manufacturing, high technology manufacturing, software]"
4,be9238b6-b07c-4ae4-b254-90f835fcdd1f,"[Adaptive Services, Multidimensional Service Innovators]","[consumer oriented services, other business services, skill-intensive services]"
5,d0d2e3a9-4e86-4f03-88e5-a906caee25b8,[Belum dapat menentukan cluster yang sesuai.],"[wholesale and retail market, other business services, skill-intensive services]"
6,d2710fef-ae5e-4c41-85ab-deb4645fd739,"[Strategic Risk Navigators, Adaptive Services, Focused Tech Innovators]","[skill-intensive services, other business services, construction]"
7,bd6c302c-be1e-4c54-95c3-f57962e32582,"[Focused Tech Innovators, Dynamic Knowledge Innovators]","[cutting edge technology manufacturing, high technology manufacturing, software]"
8,58ce9430-163d-477f-b327-38a2d8102668,"[Dynamic Knowledge Innovators, Focused Tech Innovators]","[cutting edge technology manufacturing, high technology manufacturing, software]"


## CODE TRACKING

In [ ]:
def get_recommended_cluster_refined_v4(row):
    log = []

    qA = row.get("cluster_top5_best_qA", [])
    qB = row.get("cluster_top5_best_qB", [])
    mapping = row.get("top5_cluster_to_sector_qA", {}) or {}
    rankingA = row.get("cluster_top5_ranking_qA", [])

    # Normalisasi
    if isinstance(qA, str):
        qA = [x.strip() for x in qA.split(",")] if qA.strip() else []
    if isinstance(qB, str):
        qB = [x.strip() for x in qB.split(",")] if qB.strip() else []

    log.append(f"Q-A list: {qA}")
    log.append(f"Q-B list: {qB}")

    if not qA or not qB:
        result = ["Belum dapat menentukan cluster yang sesuai."]
        log.append("Rule 3 karena salah satu kosong.")
        return result, "\n".join(log)

    # RULE 1 — irisannya
    intersection = [c for c in qA if c in qB]
    if intersection:
        log.append("Rule 1 — Ada irisan.")
        initial = intersection[:]
    else:
        log.append("Rule 2 — Tidak ada irisan.")
        initial = [qA[0], qB[0]]
        log.append(f"Kandidat awal Rule 2: {initial}")

    log.append("\n--- Tahap Validasi Kandidat Initial ---")

    valid_initial = []
    invalid_initial = []

    for c in initial:
        if c in mapping and len(mapping.get(c, [])) > 0:
            valid_initial.append(c)
        else:
            invalid_initial.append(c)

    log.append(f"Valid initial clusters: {valid_initial}")
    log.append(f"Invalid initial clusters: {invalid_initial}")

    final_candidates = valid_initial.copy()

    # Kandidat yang tersedia untuk substitusi
    all_clusters = list(dict.fromkeys(qA + qB))

    def sektor_baru_dari(cluster, sektor_terwakili):
        return set(mapping.get(cluster, [])) - sektor_terwakili

    # Substitusi hanya untuk invalid
    for invalid in invalid_initial:
        log.append(f"\nSubstitusi untuk cluster invalid: {invalid}")

        sektor_terwakili = set()
        for c in final_candidates:
            sektor_terwakili.update(mapping.get(c, []))

        best_choice = None
        best_new_sector = -1

        for candidate in all_clusters:
            if candidate == invalid:
                continue
            if candidate in final_candidates:
                continue
            if candidate not in mapping:
                continue

            new_sector_count = len(sektor_baru_dari(candidate, sektor_terwakili))

            if new_sector_count > best_new_sector:
                best_new_sector = new_sector_count
                best_choice = candidate

        if best_choice:
            log.append(f" → Diganti dengan: {best_choice} (menambah {best_new_sector} sektor baru)")
            final_candidates.append(best_choice)
        else:
            log.append(" → Tidak ada substitusi yang valid.")

    # --------------------------------------------------------------------
    # ⭐ NEW RULE: Tambah cluster jika masih ada sektor user yang belum tercover
    # --------------------------------------------------------------------
    sektor_user = set(row.get("top3_euclid_qA", []))

    sektor_terwakili = set()
    for c in final_candidates:
        sektor_terwakili.update(mapping.get(c, []))

    sektor_kurang = sektor_user - sektor_terwakili

    if sektor_kurang:
        log.append(f"Ada sektor yang belum terwakili: {sektor_kurang}")

        best_choice = None
        best_gain = -1

        for candidate in all_clusters:
            if candidate in final_candidates:
                continue
            if candidate not in mapping:
                continue

            sektor_candidate = set(mapping[candidate])
            gain = len(sektor_candidate & sektor_kurang)

            if gain > best_gain:
                best_gain = gain
                best_choice = candidate

        if best_choice:
            log.append(f"Menambahkan cluster kedua karena sektor kurang → {best_choice} (gain={best_gain})")
            final_candidates.append(best_choice)
        else:
            log.append("Tidak ada cluster tambahan yang bisa menutupi sektor yang kurang.")

    log.append(f"\nFinal Result: {final_candidates}")
    return final_candidates, "\n".join(log)

In [ ]:
recommended, log = get_recommended_cluster_refined_v4(df.iloc[0])

print("RECOMMENDED:", recommended)
print("\nLOG:", log)

print("\n")
print(df['top3_euclid_qA'][0])

print("\n")
print(df['cluster_top5_ranking_qA'][0])

print("\n")
df['top5_cluster_to_sector_qA'][0]

RECOMMENDED: ['Adaptive Services', 'Multidimensional Service Innovators']

LOG: Q-A list: ['Adaptive Services']
Q-B list: ['Adaptive Services', 'Multidimensional Service Innovators']
Rule 1 — Ada irisan.

--- Tahap Validasi Kandidat Initial ---
Valid initial clusters: ['Adaptive Services']
Invalid initial clusters: []
Ada sektor yang belum terwakili: {'skill-intensive services'}
Menambahkan cluster kedua karena sektor kurang → Multidimensional Service Innovators (gain=1)

Final Result: ['Adaptive Services', 'Multidimensional Service Innovators']


['wholesale and retail market', 'other business services', 'skill-intensive services']


[('Adaptive Services', 9), ('Multidimensional Service Innovators', 8), ('Strategic Risk Navigators', 7), ('Focused Tech Innovators', 4), ('Dynamic Knowledge Innovators', 3)]




{'Adaptive Services': ['wholesale and retail market',
  'other business services'],
 'Dynamic Knowledge Innovators': ['skill-intensive services'],
 'Strategic Risk Navigators': ['other business services',
  'skill-intensive services'],
 'Multidimensional Service Innovators': ['wholesale and retail market',
  'skill-intensive services'],
 'Focused Tech Innovators': ['other business services']}

In [ ]:
recommended, log = get_recommended_cluster_refined_v4(df.iloc[1])

print("RECOMMENDED:", recommended)
print("\nLOG:", log)

print("\n")
print(df['top3_euclid_qA'][1])

print("\n")
print(df['cluster_top5_ranking_qA'][1])

print("\n")
df['top5_cluster_to_sector_qA'][1]

RECOMMENDED: ['Adaptive Services', 'Multidimensional Service Innovators']

LOG: Q-A list: ['Adaptive Services']
Q-B list: ['Multidimensional Service Innovators']
Rule 2 — Tidak ada irisan.
Kandidat awal Rule 2: ['Adaptive Services', 'Multidimensional Service Innovators']

--- Tahap Validasi Kandidat Initial ---
Valid initial clusters: ['Adaptive Services', 'Multidimensional Service Innovators']
Invalid initial clusters: []

Final Result: ['Adaptive Services', 'Multidimensional Service Innovators']


['consumer oriented services', 'other business services', 'skill-intensive services']


[('Adaptive Services', 9), ('Multidimensional Service Innovators', 8), ('Strategic Risk Navigators', 7), ('Focused Tech Innovators', 4), ('Dynamic Knowledge Innovators', 3)]




{'Adaptive Services': ['consumer oriented services',
  'other business services'],
 'Dynamic Knowledge Innovators': ['skill-intensive services'],
 'Strategic Risk Navigators': ['other business services',
  'skill-intensive services'],
 'Multidimensional Service Innovators': ['consumer oriented services',
  'skill-intensive services'],
 'Focused Tech Innovators': ['other business services']}

In [ ]:
recommended, log = get_recommended_cluster_refined_v4(df.iloc[2])

print("RECOMMENDED:", recommended)
print("\nLOG:", log)

print("\n")
print(df['top3_euclid_qA'][2])

print("\n")
print(df['cluster_top5_ranking_qA'][2])

print("\n")
df['top5_cluster_to_sector_qA'][2]

RECOMMENDED: ['Multidimensional Service Innovators']

LOG: Q-A list: ['Multidimensional Service Innovators']
Q-B list: ['Focused Tech Innovators']
Rule 2 — Tidak ada irisan.
Kandidat awal Rule 2: ['Multidimensional Service Innovators', 'Focused Tech Innovators']

--- Tahap Validasi Kandidat Initial ---
Valid initial clusters: ['Multidimensional Service Innovators']
Invalid initial clusters: ['Focused Tech Innovators']

Substitusi untuk cluster invalid: Focused Tech Innovators
 → Tidak ada substitusi yang valid.
Ada sektor yang belum terwakili: {'cutting edge technology manufacturing'}
Tidak ada cluster tambahan yang bisa menutupi sektor yang kurang.

Final Result: ['Multidimensional Service Innovators']


['consumer oriented services', 'skill-intensive services', 'cutting edge technology manufacturing']


[('Multidimensional Service Innovators', 9), ('Adaptive Services', 5), ('Dynamic Knowledge Innovators', 4), ('Strategic Risk Navigators', 4), ('Focused Tech Innovators', 0)]




{'Adaptive Services': ['consumer oriented services'],
 'Dynamic Knowledge Innovators': ['skill-intensive services'],
 'Strategic Risk Navigators': ['skill-intensive services'],
 'Multidimensional Service Innovators': ['consumer oriented services',
  'skill-intensive services']}

In [ ]:
recommended, log = get_recommended_cluster_refined_v4(df.iloc[3])

print("RECOMMENDED:", recommended)
print("\nLOG:", log)

print("\n")
print(df['top3_euclid_qA'][3])

print("\n")
print(df['cluster_top5_ranking_qA'][3])

print("\n")
df['top5_cluster_to_sector_qA'][3]

RECOMMENDED: ['Dynamic Knowledge Innovators', 'Focused Tech Innovators']

LOG: Q-A list: ['Dynamic Knowledge Innovators', 'Focused Tech Innovators']
Q-B list: ['Dynamic Knowledge Innovators', 'Strategic Risk Navigators', 'Multidimensional Service Innovators']
Rule 1 — Ada irisan.

--- Tahap Validasi Kandidat Initial ---
Valid initial clusters: ['Dynamic Knowledge Innovators']
Invalid initial clusters: []
Ada sektor yang belum terwakili: {'cutting edge technology manufacturing'}
Menambahkan cluster kedua karena sektor kurang → Focused Tech Innovators (gain=0)

Final Result: ['Dynamic Knowledge Innovators', 'Focused Tech Innovators']


['cutting edge technology manufacturing', 'high technology manufacturing', 'software']


[('Dynamic Knowledge Innovators', 7), ('Focused Tech Innovators', 7), ('Adaptive Services', 0), ('Strategic Risk Navigators', 0), ('Multidimensional Service Innovators', 0)]




{'Dynamic Knowledge Innovators': ['high technology manufacturing', 'software'],
 'Focused Tech Innovators': ['high technology manufacturing', 'software']}

In [ ]:
recommended, log = get_recommended_cluster_refined_v4(df.iloc[4])

print("RECOMMENDED:", recommended)
print("\nLOG:", log)

print("\n")
print(df['top3_euclid_qA'][4])

print("\n")
print(df['cluster_top5_ranking_qA'][4])

print("\n")
df['top5_cluster_to_sector_qA'][4]

RECOMMENDED: ['Adaptive Services', 'Multidimensional Service Innovators']

LOG: Q-A list: ['Adaptive Services']
Q-B list: ['Adaptive Services', 'Multidimensional Service Innovators', 'Focused Tech Innovators']
Rule 1 — Ada irisan.

--- Tahap Validasi Kandidat Initial ---
Valid initial clusters: ['Adaptive Services']
Invalid initial clusters: []
Ada sektor yang belum terwakili: {'skill-intensive services'}
Menambahkan cluster kedua karena sektor kurang → Multidimensional Service Innovators (gain=1)

Final Result: ['Adaptive Services', 'Multidimensional Service Innovators']


['consumer oriented services', 'other business services', 'skill-intensive services']


[('Adaptive Services', 9), ('Multidimensional Service Innovators', 8), ('Strategic Risk Navigators', 7), ('Focused Tech Innovators', 4), ('Dynamic Knowledge Innovators', 3)]




{'Adaptive Services': ['consumer oriented services',
  'other business services'],
 'Dynamic Knowledge Innovators': ['skill-intensive services'],
 'Strategic Risk Navigators': ['other business services',
  'skill-intensive services'],
 'Multidimensional Service Innovators': ['consumer oriented services',
  'skill-intensive services'],
 'Focused Tech Innovators': ['other business services']}

In [ ]:
recommended, log = get_recommended_cluster_refined_v4(df.iloc[5])

print("RECOMMENDED:", recommended)
print("\nLOG:", log)

print("\n")
print(df['top3_euclid_qA'][5])

print("\n")
print(df['cluster_top5_ranking_qA'][5])

print("\n")
df['top5_cluster_to_sector_qA'][5]

RECOMMENDED: ['Belum dapat menentukan cluster yang sesuai.']

LOG: Q-A list: ['Adaptive Services']
Q-B list: []
Rule 3 karena salah satu kosong.


['wholesale and retail market', 'other business services', 'skill-intensive services']


[('Adaptive Services', 9), ('Multidimensional Service Innovators', 8), ('Strategic Risk Navigators', 7), ('Focused Tech Innovators', 4), ('Dynamic Knowledge Innovators', 3)]




{'Adaptive Services': ['wholesale and retail market',
  'other business services'],
 'Dynamic Knowledge Innovators': ['skill-intensive services'],
 'Strategic Risk Navigators': ['other business services',
  'skill-intensive services'],
 'Multidimensional Service Innovators': ['wholesale and retail market',
  'skill-intensive services'],
 'Focused Tech Innovators': ['other business services']}

In [ ]:
recommended, log = get_recommended_cluster_refined_v4(df.iloc[6])

print("RECOMMENDED:", recommended)
print("\nLOG:", log)

print("\n")
print(df['top3_euclid_qA'][6])

print("\n")
print(df['cluster_top5_ranking_qA'][6])

print("\n")
df['top5_cluster_to_sector_qA'][6]

RECOMMENDED: ['Strategic Risk Navigators', 'Adaptive Services', 'Focused Tech Innovators']

LOG: Q-A list: ['Strategic Risk Navigators']
Q-B list: ['Adaptive Services', 'Focused Tech Innovators']
Rule 2 — Tidak ada irisan.
Kandidat awal Rule 2: ['Strategic Risk Navigators', 'Adaptive Services']

--- Tahap Validasi Kandidat Initial ---
Valid initial clusters: ['Strategic Risk Navigators', 'Adaptive Services']
Invalid initial clusters: []
Ada sektor yang belum terwakili: {'construction'}
Menambahkan cluster kedua karena sektor kurang → Focused Tech Innovators (gain=0)

Final Result: ['Strategic Risk Navigators', 'Adaptive Services', 'Focused Tech Innovators']


['skill-intensive services', 'other business services', 'construction']


[('Strategic Risk Navigators', 9), ('Multidimensional Service Innovators', 8), ('Dynamic Knowledge Innovators', 5), ('Adaptive Services', 4), ('Focused Tech Innovators', 4)]




{'Adaptive Services': ['other business services'],
 'Dynamic Knowledge Innovators': ['skill-intensive services'],
 'Strategic Risk Navigators': ['skill-intensive services',
  'other business services'],
 'Multidimensional Service Innovators': ['skill-intensive services',
  'construction'],
 'Focused Tech Innovators': ['other business services']}

In [ ]:
recommended, log = get_recommended_cluster_refined_v4(df.iloc[7])

print("RECOMMENDED:", recommended)
print("\nLOG:", log)

print("\n")
print(df['top3_euclid_qA'][7])

print("\n")
print(df['cluster_top5_ranking_qA'][7])

print("\n")
df['top5_cluster_to_sector_qA'][7]

RECOMMENDED: ['Focused Tech Innovators', 'Dynamic Knowledge Innovators']

LOG: Q-A list: ['Dynamic Knowledge Innovators', 'Focused Tech Innovators']
Q-B list: ['Adaptive Services', 'Focused Tech Innovators']
Rule 1 — Ada irisan.

--- Tahap Validasi Kandidat Initial ---
Valid initial clusters: ['Focused Tech Innovators']
Invalid initial clusters: []
Ada sektor yang belum terwakili: {'cutting edge technology manufacturing'}
Menambahkan cluster kedua karena sektor kurang → Dynamic Knowledge Innovators (gain=0)

Final Result: ['Focused Tech Innovators', 'Dynamic Knowledge Innovators']


['cutting edge technology manufacturing', 'high technology manufacturing', 'software']


[('Dynamic Knowledge Innovators', 7), ('Focused Tech Innovators', 7), ('Adaptive Services', 0), ('Strategic Risk Navigators', 0), ('Multidimensional Service Innovators', 0)]




{'Dynamic Knowledge Innovators': ['high technology manufacturing', 'software'],
 'Focused Tech Innovators': ['high technology manufacturing', 'software']}

In [ ]:
recommended, log = get_recommended_cluster_refined_v4(df.iloc[8])

print("RECOMMENDED:", recommended)
print("\nLOG:", log)

print("\n")
print(df['top3_euclid_qA'][8])

print("\n")
print(df['cluster_top5_ranking_qA'][8])

print("\n")
df['top5_cluster_to_sector_qA'][8]

RECOMMENDED: ['Dynamic Knowledge Innovators', 'Focused Tech Innovators']

LOG: Q-A list: ['Dynamic Knowledge Innovators', 'Focused Tech Innovators']
Q-B list: ['Multidimensional Service Innovators']
Rule 2 — Tidak ada irisan.
Kandidat awal Rule 2: ['Dynamic Knowledge Innovators', 'Multidimensional Service Innovators']

--- Tahap Validasi Kandidat Initial ---
Valid initial clusters: ['Dynamic Knowledge Innovators']
Invalid initial clusters: ['Multidimensional Service Innovators']

Substitusi untuk cluster invalid: Multidimensional Service Innovators
 → Diganti dengan: Focused Tech Innovators (menambah 0 sektor baru)
Ada sektor yang belum terwakili: {'cutting edge technology manufacturing'}
Tidak ada cluster tambahan yang bisa menutupi sektor yang kurang.

Final Result: ['Dynamic Knowledge Innovators', 'Focused Tech Innovators']


['cutting edge technology manufacturing', 'high technology manufacturing', 'software']


[('Dynamic Knowledge Innovators', 7), ('Focused Tech Innovators', 7), 

{'Dynamic Knowledge Innovators': ['high technology manufacturing', 'software'],
 'Focused Tech Innovators': ['high technology manufacturing', 'software']}